In [28]:
from pathlib import Path
import numpy as np
# import h5py
from tqdm.auto import tqdm, trange
import scipy.io
import time
import torch
import random
import math
import shutil
from sklearn.decomposition import PCA
import scipy as sp
import scipy.signal
import os
import pandas as pd
from torch import nn
from torchvision import transforms, datasets
from torch.utils.data import Dataset
from scipy.spatial.distance import cdist
# import spikeinterface.core as sc
# import spikeinterface.full as si
from typing import Any, Callable, Optional, Tuple

from analysis.projections import pca_train, pca, get_flattened_data
from analysis.plotting import plot_gmm, plot_closest_spikes
from analysis.encoder_utils import load_GPT_backbone, get_fcenc_backbone
from analysis.cluster import GMM, HDBSCAN
from sklearn.mixture import GaussianMixture
from sklearn.metrics import adjusted_rand_score
from analysis.cluster import MeanShift
from analysis.benchmarking import class_scores, avg_score, per_class_accs, avg_class_accs
import matplotlib.patheffects as pe

from ceed.models.model_simclr import FullyConnectedEnc
from utils.utils import gmm_monitor, knn_monitor
from data_aug.wf_data_augs import Crop
from data_aug.contrastive_learning_dataset import WFDataset_lab 

In [29]:
# class Args(Object):
#     def __init__(epoch_nums=1):
#         epochs = epoch_nums
    
class Args(object):
    pass

args = Args()
args.epochs = 1
args.use_chan_pos = False
args.use_gpt = False
print(args.epochs)

1


In [83]:
ten_neur_id_path_dy016 = '/Users/ankit/Documents/PaninskiLab/contrastive_spikes/dy016_random_choice_fix_mc'
ten_neur_id_unshifted_path_dy016 = '/Users/ankit/Documents/PaninskiLab/contrastive_spikes/dy016_10n_1200s_pcaunshifted'
ten_neur_ood_path_dy016 = '/Users/ankit/Documents/PaninskiLab/contrastive_spikes/dy016_10_neuron_400ood'
ten_neur_ood_path_dy009 = '/Users/ankit/Documents/PaninskiLab/contrastive_spikes/dy009_10_neuron_400ood'
fourhund_neur_path = '/Users/ankit/Documents/PaninskiLab/contrastive_spikes/real400n_200s'
fourhund_neur_1200_path = '/Users/ankit/Documents/PaninskiLab/contrastive_spikes/real400n_1200s'
threenine_neur_1200_path = '/Users/ankit/Documents/PaninskiLab/contrastive_spikes/real390n_1200s'
threenine_neur_200_path = '/Users/ankit/Documents/PaninskiLab/contrastive_spikes/real390n_200s'
sixhund_neur_path = '/Users/ankit/Documents/PaninskiLab/contrastive_spikes/real600n_1200s'
test_setA = '/Users/ankit/Documents/PaninskiLab/contrastive_spikes/dy016_og_goodunit_subset'
test_setA_unshifted = '/Users/ankit/Documents/PaninskiLab/contrastive_spikes/dy016_10n_200s_pcaunshifted'
test_set_zfm = '/Users/ankit/Documents/PaninskiLab/contrastive_spikes/zfm_10n_OOD_set_hp'
test_fn = 'spikes_test.npy' 
train_fn = 'spikes_train.npy'

single_chan_ckpt = '/Users/ankit/Documents/PaninskiLab/spike-psvae/pretrained/single_chan_denoiser.pt'

In [84]:
# class WFDataset_lab(Dataset):
#     train_set_fn = "spikes_train.npy"
#     train_targets_fn = "labels_train.npy"
#     spike_mcs_train_fn = "channel_num_train.npy"
#     chan_coords_train_fn = "channel_spike_locs_train.npy"

#     test_set_fn = "spikes_test.npy"
#     test_targets_fn = "labels_test.npy"
#     spike_mcs_test_fn = "channel_num_test.npy"
#     chan_coords_test_fn = "channel_spike_locs_test.npy"

#     val_set_fn = "spikes_val.npy"
#     val_targets_fn = "labels_val.npy"
#     spike_mcs_val_fn = "channel_num_val.npy"
#     chan_coords_val_fn = "channel_spike_locs_val.npy"
    
#     def __init__(
#         self,
#         root: str,
#         use_chan_pos: bool = False, 
#         multi_chan: bool = False,
#         split: str = 'train',
#         transform: Optional[Callable] = None,
        
#     ) -> None:

#         super().__init__()
#         if split == 'train':
#             print(multi_chan, self.train_set_fn)
#             self.data = np.load(os.path.join(root, self.train_set_fn)).astype('float32')
#             self.targets = np.load(os.path.join(root, self.train_targets_fn))
#             self.chan_nums = np.load(os.path.join(root, self.spike_mcs_train_fn))
#             self.channel_locs = np.load(os.path.join(root, self.chan_coords_train_fn))
#         elif split == 'test':
#             self.data = np.load(os.path.join(root, self.test_set_fn)).astype('float32')
#             self.targets = np.load(os.path.join(root, self.test_targets_fn))
#             self.chan_nums = np.load(os.path.join(root, self.spike_mcs_test_fn))
#             self.channel_locs = np.load(os.path.join(root, self.chan_coords_test_fn))
#         elif split == 'val':
#             self.data = np.load(os.path.join(root, self.val_set_fn)).astype('float32')
#             self.targets = np.load(os.path.join(root, self.val_targets_fn))
#             self.chan_nums = np.load(os.path.join(root, self.spike_mcs_val_fn))
#             self.channel_locs = np.load(os.path.join(root, self.chan_coords_val_fn))
            
#         # self.data: Any = []

#         # now load the numpy array
#         print(self.data.shape)
#         self.root = root
#         self.transform = transform
#         self.use_chan_pos = use_chan_pos
#         self.num_classes = len(np.unique(self.targets))

#     def __getitem__(self, index: int) -> Any :
#         """
#         Args:
#             index (int): Index
#         Returns:
#             tensor: wf
#         """
#         wf = self.data[index].astype('float32')
#         y = self.targets[index].astype('long')
#         chan_nums = self.chan_nums[index]
#         chan_loc = self.channel_locs[index].astype('float32')

#         # doing this so that it is a tensor
#         # wf = torch.from_numpy(wf)
#         if self.transform is not None and self.use_chan_pos:
#             wf, chan_loc = self.transform([wf, chan_nums, chan_loc])
#         elif self.transform is not None:
#             wf = self.transform([wf, chan_nums, y])

#         if self.use_chan_pos:
#             return [wf, chan_loc], y

#         return wf, y

#     def __len__(self) -> int:
#         return len(self.data)
    

# class UnCrop(object):
#     def __init__(self, num_extra_chans=2, ignore_chan_num=True, unshift=True, max_chans=None):
#         self.num_extra_chans = num_extra_chans
#         self.ignore_chan_num = ignore_chan_num
#         self.unshift = unshift
#         self.max_chans = max_chans
        
#     def __call__(self, sample):
#         if len(sample) == 3:
#             wf, chan_nums, y = sample
#         else:
#             wf, chan_nums = sample
        
#         if len(wf.shape) == 1:
#             wf = np.expand_dims(wf, axis=0)
#         n_chans = wf.shape[0]
#         n_times = wf.shape[1]
        
#         max_chan_ind = math.floor(n_chans/2)

#         if self.unshift:
#             curr_mc = self.max_chans[y]
#             max_chan_ind = list(chan_nums).index(curr_mc)
#         wf = wf[max_chan_ind-self.num_extra_chans:max_chan_ind+self.num_extra_chans+1]
# #         if wf.shape[0] != 2*self.num_extra_chans + 1:
#         if y == 1:
#             print(y)
#             print(chan_nums)
#             print(max_chan_ind)
#             print(wf.shape)

#         if not self.ignore_chan_num:
#             if type(chan_nums) != np.int64: 
#                 chan_nums = chan_nums[max_chan_ind-self.num_extra_chans:max_chan_ind+self.num_extra_chans+1]

#         # in single channel case the wf will become 1 dimensional
#         if len(wf.shape) == 1:
#             wf = np.expand_dims(wf, axis=0)
        
#         return wf

In [85]:
# dy016 ID train data
# dy016_id_5c_train_dataset = WFDataset_lab(ten_neur_id_path_dy016, split='train', multi_chan=True, transform=Crop(prob=0.0, num_extra_chans=2, ignore_chan_num=True), use_chan_pos=False)
# dy016_id_5c_train_data, dy016_id_5c_train_labels = get_flattened_data(dy016_id_5c_train_dataset)
# dy016_denoised_id_5c_train_data, dy016_denoised_id_5c_train_labels = get_flattened_data(dy016_id_5c_train_dataset, denoise=True, denoise_path=single_chan_ckpt)

# dy016_id_11c_train_dataset = WFDataset_lab(ten_neur_id_path_dy016, split='train', multi_chan=True, transform=Crop(prob=0.0, num_extra_chans=5, ignore_chan_num=True), use_chan_pos=False)
# dy016_id_11c_train_data, dy016_id_11c_train_labels = get_flattened_data(dy016_id_11c_train_dataset)
# dy016_denoised_id_11c_train_data, dy016_denoised_id_11c_train_labels = get_flattened_data(dy016_id_11c_train_dataset, denoise=True, denoise_path=single_chan_ckpt)

# dy016 ID train data (Unshifted for PCA)
# dy016_unshifted_id_3c_train_dataset = WFDataset_lab(ten_neur_id_unshifted_path_dy016, split='train', multi_chan=True, transform=Crop(prob=0.0, num_extra_chans=1, ignore_chan_num=True), use_chan_pos=False)
# dy016_unshifted_id_3c_train_data, dy016_unshifted_id_3c_train_labels = get_flattened_data(dy016_unshifted_id_3c_train_dataset)
# dy016_unshifted_denoised_id_3c_train_data, dy016_unshifted_denoised_id_3c_train_labels = get_flattened_data(dy016_unshifted_id_3c_train_dataset, denoise=True, denoise_path=single_chan_ckpt)

# dy016_unshifted_id_5c_train_dataset = WFDataset_lab(ten_neur_id_unshifted_path_dy016, split='train', multi_chan=True, transform=Crop(prob=0.0, num_extra_chans=2, ignore_chan_num=True), use_chan_pos=False)
# dy016_unshifted_id_5c_train_data, dy016_unshifted_id_5c_train_labels = get_flattened_data(dy016_unshifted_id_5c_train_dataset)
# dy016_unshifted_denoised_id_5c_train_data, dy016_unshifted_denoised_id_5c_train_labels = get_flattened_data(dy016_unshifted_id_5c_train_dataset, denoise=True, denoise_path=single_chan_ckpt)

# dy016_unshifted_id_11c_train_dataset = WFDataset_lab(ten_neur_id_unshifted_path_dy016, split='train', multi_chan=True, transform=Crop(prob=0.0, num_extra_chans=5, ignore_chan_num=True), use_chan_pos=False)
# dy016_unshifted_id_11c_train_data, dy016_unshifted_id_11c_train_labels = get_flattened_data(dy016_unshifted_id_11c_train_dataset)
# dy016_unshifted_denoised_id_11c_train_data, dy016_unshifted_denoised_id_11c_train_labels = get_flattened_data(dy016_unshifted_id_11c_train_dataset, denoise=True, denoise_path=single_chan_ckpt)

# 400 neuron, 200 spikes train data
# fourhund_5c_train_dataset = WFDataset_lab(fourhund_neur_path, split='train', multi_chan=True, transform=Crop(prob=0.0, num_extra_chans=2, ignore_chan_num=True), use_chan_pos=False)
# fourhund_5c_train_data, fourhund_5c_train_labels = get_flattened_data(fourhund_5c_train_dataset)
# fourhund_denoised_5c_train_data, fourhund_denoised_5c_train_labels = get_flattened_data(fourhund_5c_train_dataset, denoise=True, denoise_path=single_chan_ckpt)

# fourhund_11c_train_dataset = WFDataset_lab(fourhund_neur_path, split='train', multi_chan=True, transform=Crop(prob=0.0, num_extra_chans=5, ignore_chan_num=True), use_chan_pos=False)
# fourhund_11c_train_data, fourhund_11c_train_labels = get_flattened_data(fourhund_11c_train_dataset)
# fourhund_denoised_11c_train_data, fourhund_denoised_11c_train_labels = get_flattened_data(fourhund_11c_train_dataset, denoise=True, denoise_path=single_chan_ckpt)

# 400 neuron, 1200 spikes train data
# fourhund_1200_5c_train_dataset = WFDataset_lab(fourhund_neur_1200_path, split='train', multi_chan=True, transform=Crop(prob=0.0, num_extra_chans=2, ignore_chan_num=True), use_chan_pos=False)
# fourhund_1200_5c_train_data, fourhund_1200_5c_train_labels = get_flattened_data(fourhund_1200_5c_train_dataset)
# fourhund_1200_denoised_5c_train_data, fourhund_1200_denoised_5c_train_labels = get_flattened_data(fourhund_1200_5c_train_dataset, denoise=True, denoise_path=single_chan_ckpt)

# fourhund_1200_11c_train_dataset = WFDataset_lab(fourhund_neur_1200_path, split='train', multi_chan=True, transform=Crop(prob=0.0, num_extra_chans=5, ignore_chan_num=True), use_chan_pos=False)
# fourhund_1200_11c_train_data, fourhund_1200_11c_train_labels = get_flattened_data(fourhund_1200_11c_train_dataset)
# fourhund_1200_denoised_11c_train_data, fourhund_1200_denoised_11c_train_labels = get_flattened_data(fourhund_1200_11c_train_dataset, denoise=True, denoise_path=single_chan_ckpt)

# 390 neuron, 1200 spikes train data
# threenine_1200_5c_train_dataset = WFDataset_lab(threenine_neur_1200_path, split='train', multi_chan=True, transform=Crop(prob=0.0, num_extra_chans=2, ignore_chan_num=True), use_chan_pos=False)
# threenine_1200_5c_train_data, threenine_1200_5c_train_labels = get_flattened_data(threenine_1200_5c_train_dataset)
# threenine_1200_denoised_5c_train_data, threenine_1200_denoised_5c_train_labels = get_flattened_data(threenine_1200_5c_train_dataset, denoise=True, denoise_path=single_chan_ckpt)

# threenine_1200_11c_train_dataset = WFDataset_lab(threenine_neur_1200_path, split='train', multi_chan=True, transform=Crop(prob=0.0, num_extra_chans=5, ignore_chan_num=True), use_chan_pos=False)
# threenine_1200_11c_train_data, threenine_1200_11c_train_labels = get_flattened_data(threenine_1200_11c_train_dataset)
# threenine_1200_denoised_11c_train_data, threenine_1200_denoised_11c_train_labels = get_flattened_data(threenine_1200_11c_train_dataset, denoise=True, denoise_path=single_chan_ckpt)

# 390 neuron, 1200 spikes train data
# threenine_200_5c_train_dataset = WFDataset_lab(threenine_neur_200_path, split='train', multi_chan=True, transform=Crop(prob=0.0, num_extra_chans=2, ignore_chan_num=True), use_chan_pos=False)
# threenine_200_5c_train_data, threenine_200_5c_train_labels = get_flattened_data(threenine_200_5c_train_dataset)
# threenine_200_denoised_5c_train_data, threenine_200_denoised_5c_train_labels = get_flattened_data(threenine_200_5c_train_dataset, denoise=True, denoise_path=single_chan_ckpt)

# threenine_200_11c_train_dataset = WFDataset_lab(threenine_neur_200_path, split='train', multi_chan=True, transform=Crop(prob=0.0, num_extra_chans=5, ignore_chan_num=True), use_chan_pos=False)
# threenine_200_11c_train_data, threenine_200_11c_train_labels = get_flattened_data(threenine_200_11c_train_dataset)
# threenine_200_denoised_11c_train_data, threenine_200_denoised_11c_train_labels = get_flattened_data(threenine_200_11c_train_dataset, denoise=True, denoise_path=single_chan_ckpt)

# dy016 ood data loaders
# dy016_ood_5c_memory_dataset = WFDataset_lab(ten_neur_ood_path_dy016, split='train', multi_chan=True, transform=Crop(prob=0.0, num_extra_chans=2, ignore_chan_num=True), use_chan_pos=False)
# dy016_ood_5c_memory_data, dy016_ood_5c_memory_labels = get_flattened_data(dy016_ood_5c_memory_dataset)
# dy016_denoised_ood_5c_memory_data, dy016_denoised_ood_5c_memory_labels = get_flattened_data(dy016_ood_5c_memory_dataset, denoise=True, denoise_path=single_chan_ckpt)

# dy016_ood_5c_test_dataset = WFDataset_lab(ten_neur_ood_path_dy016, split='test', multi_chan=True, transform=Crop(prob=0.0, num_extra_chans=2, ignore_chan_num=True), use_chan_pos=False)
# dy016_ood_5c_test_data, dy016_ood_5c_test_labels = get_flattened_data(dy016_ood_5c_test_dataset)
# dy016_denoised_ood_5c_test_data, dy016_denoised_ood_5c_test_labels = get_flattened_data(dy016_ood_5c_test_dataset, denoise=True, denoise_path=single_chan_ckpt)

# dy016_ood_11c_memory_dataset = WFDataset_lab(ten_neur_ood_path_dy016, split='train', multi_chan=True, transform=Crop(prob=0.0, num_extra_chans=5, ignore_chan_num=True), use_chan_pos=False)
# dy016_ood_11c_memory_data, dy016_ood_11c_memory_labels = get_flattened_data(dy016_ood_11c_memory_dataset)
# dy016_denoised_ood_11c_memory_data, dy016_denoised_ood_11c_memory_labels = get_flattened_data(dy016_ood_11c_memory_dataset, denoise=True, denoise_path=single_chan_ckpt)

# dy016_ood_11c_test_dataset = WFDataset_lab(ten_neur_ood_path_dy016, split='test', multi_chan=True, transform=Crop(prob=0.0, num_extra_chans=5, ignore_chan_num=True), use_chan_pos=False)
# dy016_ood_11c_test_data, dy016_ood_11c_test_labels = get_flattened_data(dy016_ood_11c_test_dataset)
# dy016_denoised_ood_11c_test_data, dy016_denoised_ood_11c_test_labels = get_flattened_data(dy016_ood_11c_test_dataset, denoise=True, denoise_path=single_chan_ckpt)

# dy009 ood data loaders
# dy009_ood_5c_memory_dataset = WFDataset_lab(ten_neur_ood_path_dy009, split='train', multi_chan=True, transform=Crop(prob=0.0, num_extra_chans=2, ignore_chan_num=True), use_chan_pos=False)
# dy009_ood_5c_memory_data, dy009_ood_5c_memory_labels = get_flattened_data(dy009_ood_5c_memory_dataset)
# dy009_denoised_ood_5c_memory_data, dy009_denoised_ood_5c_memory_labels = get_flattened_data(dy009_ood_5c_memory_dataset, denoise=True, denoise_path=single_chan_ckpt)

# dy009_ood_5c_test_dataset = WFDataset_lab(ten_neur_ood_path_dy009, split='test', multi_chan=True, transform=Crop(prob=0.0, num_extra_chans=2, ignore_chan_num=True), use_chan_pos=False)
# dy009_ood_5c_test_data, dy009_ood_5c_test_labels = get_flattened_data(dy016_ood_5c_test_dataset)
# dy009_denoised_ood_5c_test_data, dy009_denoised_ood_5c_test_labels = get_flattened_data(dy009_ood_5c_memory_dataset, denoise=True, denoise_path=single_chan_ckpt)

# dy009_ood_11c_memory_dataset = WFDataset_lab(ten_neur_ood_path_dy009, split='train', multi_chan=True, transform=Crop(prob=0.0, num_extra_chans=5, ignore_chan_num=True), use_chan_pos=False)
# dy009_ood_11c_memory_data, dy009_ood_11c_memory_labels = get_flattened_data(dy009_ood_11c_memory_dataset)
# dy009_denoised_ood_11c_memory_data, dy009_denoised_ood_11c_memory_labels = get_flattened_data(dy009_ood_11c_memory_dataset, denoise=True, denoise_path=single_chan_ckpt)

# dy009_ood_11c_test_dataset = WFDataset_lab(ten_neur_ood_path_dy009, split='test', multi_chan=True, transform=Crop(prob=0.0, num_extra_chans=5, ignore_chan_num=True), use_chan_pos=False)
# dy009_ood_11c_test_data, dy009_ood_11c_test_labels = get_flattened_data(dy009_ood_11c_test_dataset)
# dy009_denoised_ood_11c_test_data, dy009_denoised_ood_11c_test_labels = get_flattened_data(dy009_ood_11c_test_dataset, denoise=True, denoise_path=single_chan_ckpt)

# ten neuron ID data loaders
# dy009 ood data loaders
# id_5c_memory_dataset = WFDataset_lab(test_setA, split='train', multi_chan=True, transform=Crop(prob=0.0, num_extra_chans=2, ignore_chan_num=True), use_chan_pos=False)
# id_5c_memory_data, id_5c_memory_labels = get_flattened_data(id_5c_memory_dataset)
# id_denoised_5c_memory_data, id_denoised_5c_memory_labels = get_flattened_data(id_5c_memory_dataset, denoise=True, denoise_path=single_chan_ckpt)

# id_5c_test_dataset = WFDataset_lab(test_setA, split='test', multi_chan=True, transform=Crop(prob=0.0, num_extra_chans=2, ignore_chan_num=True), use_chan_pos=False)
# id_5c_test_data, id_5c_test_labels = get_flattened_data(id_5c_test_dataset)
# id_denoised_5c_test_data, id_denoised_5c_test_labels = get_flattened_data(id_5c_test_dataset, denoise=True, denoise_path=single_chan_ckpt)

# id_11c_memory_dataset = WFDataset_lab(test_setA, split='train', multi_chan=True, transform=Crop(prob=0.0, num_extra_chans=5, ignore_chan_num=True), use_chan_pos=False)
# id_11c_memory_data, id_11c_memory_labels = get_flattened_data(id_11c_memory_dataset)
# id_denoised_11c_memory_data, id_denoised_11c_memory_labels = get_flattened_data(id_11c_memory_dataset, denoise=True, denoise_path=single_chan_ckpt)

# id_11c_test_dataset = WFDataset_lab(test_setA, split='test', multi_chan=True, transform=Crop(prob=0.0, num_extra_chans=5, ignore_chan_num=True), use_chan_pos=False)
# id_11c_test_data, id_11c_test_labels = get_flattened_data(id_11c_test_dataset)
# id_denoised_11c_test_data, id_denoised_11c_test_labels = get_flattened_data(id_11c_test_dataset, denoise=True, denoise_path=single_chan_ckpt)

# ZFM OOD recording data loaders
zfm_5c_memory_dataset = WFDataset_lab(test_set_zfm, split='train', multi_chan=True, transform=Crop(prob=0.0, num_extra_chans=2, ignore_chan_num=True), use_chan_pos=False)
zfm_5c_memory_data, zfm_5c_memory_labels = get_flattened_data(zfm_5c_memory_dataset)
zfm_denoised_5c_memory_data, zfm_denoised_5c_memory_labels = get_flattened_data(zfm_5c_memory_dataset, denoise=True, denoise_path=single_chan_ckpt)

zfm_5c_test_dataset = WFDataset_lab(test_set_zfm, split='test', multi_chan=True, transform=Crop(prob=0.0, num_extra_chans=2, ignore_chan_num=True), use_chan_pos=False)
zfm_5c_test_data, zfm_5c_test_labels = get_flattened_data(zfm_5c_test_dataset)
zfm_denoised_5c_test_data, zfm_denoised_5c_test_labels = get_flattened_data(zfm_5c_test_dataset, denoise=True, denoise_path=single_chan_ckpt)

zfm_11c_memory_dataset = WFDataset_lab(test_set_zfm, split='train', multi_chan=True, transform=Crop(prob=0.0, num_extra_chans=5, ignore_chan_num=True), use_chan_pos=False)
zfm_11c_memory_data, zfm_11c_memory_labels = get_flattened_data(zfm_11c_memory_dataset)
zfm_denoised_11c_memory_data, zfm_denoised_11c_memory_labels = get_flattened_data(zfm_11c_memory_dataset, denoise=True, denoise_path=single_chan_ckpt)

zfm_11c_test_dataset = WFDataset_lab(test_set_zfm, split='test', multi_chan=True, transform=Crop(prob=0.0, num_extra_chans=5, ignore_chan_num=True), use_chan_pos=False)
zfm_11c_test_data, zfm_11c_test_labels = get_flattened_data(zfm_11c_test_dataset)
zfm_denoised_11c_test_data, zfm_denoised_11c_test_labels = get_flattened_data(zfm_11c_test_dataset, denoise=True, denoise_path=single_chan_ckpt)

# ten neuron 200s test ID UNSHIFTED data loaders
# id_unshifted_3c_memory_dataset = WFDataset_lab(test_setA_unshifted, split='train', multi_chan=True, transform=Crop(prob=0.0, num_extra_chans=1, ignore_chan_num=True), use_chan_pos=False)
# id_unshifted_3c_memory_data, id_unshifted_3c_memory_labels = get_flattened_data(id_unshifted_3c_memory_dataset)
# id_unshifted_denoised_3c_memory_data, id_unshifted_denoised_3c_memory_labels = get_flattened_data(id_unshifted_3c_memory_dataset, denoise=True, denoise_path=single_chan_ckpt)

# id_unshifted_3c_test_dataset = WFDataset_lab(test_setA_unshifted, split='test', multi_chan=True, transform=Crop(prob=0.0, num_extra_chans=1, ignore_chan_num=True), use_chan_pos=False)
# id_unshifted_3c_test_data, id_unshifted_3c_test_labels = get_flattened_data(id_unshifted_3c_test_dataset)
# id_unshifted_denoised_3c_test_data, id_unshifted_denoised_3c_test_labels = get_flattened_data(id_unshifted_3c_test_dataset, denoise=True, denoise_path=single_chan_ckpt)

# id_unshifted_5c_memory_dataset = WFDataset_lab(test_setA_unshifted, split='train', multi_chan=True, transform=Crop(prob=0.0, num_extra_chans=2, ignore_chan_num=True), use_chan_pos=False)
# id_unshifted_5c_memory_data, id_unshifted_5c_memory_labels = get_flattened_data(id_unshifted_5c_memory_dataset)
# id_unshifted_denoised_5c_memory_data, id_unshifted_denoised_5c_memory_labels = get_flattened_data(id_unshifted_5c_memory_dataset, denoise=True, denoise_path=single_chan_ckpt)

# id_unshifted_5c_test_dataset = WFDataset_lab(test_setA_unshifted, split='test', multi_chan=True, transform=Crop(prob=0.0, num_extra_chans=2, ignore_chan_num=True), use_chan_pos=False)
# id_unshifted_5c_test_data, id_unshifted_5c_test_labels = get_flattened_data(id_unshifted_5c_test_dataset)
# id_unshifted_denoised_5c_test_data, id_unshifted_denoised_5c_test_labels = get_flattened_data(id_unshifted_5c_test_dataset, denoise=True, denoise_path=single_chan_ckpt)

# id_unshifted_11c_memory_dataset = WFDataset_lab(test_setA_unshifted, split='train', multi_chan=True, transform=Crop(prob=0.0, num_extra_chans=5, ignore_chan_num=True), use_chan_pos=False)
# id_unshifted_11c_memory_data, id_unshifted_11c_memory_labels = get_flattened_data(id_unshifted_11c_memory_dataset)
# id_unshifted_denoised_11c_memory_data, id_unshifted_denoised_11c_memory_labels = get_flattened_data(id_unshifted_11c_memory_dataset, denoise=True, denoise_path=single_chan_ckpt)

# id_unshifted_11c_test_dataset = WFDataset_lab(test_setA_unshifted, split='test', multi_chan=True, transform=Crop(prob=0.0, num_extra_chans=5, ignore_chan_num=True), use_chan_pos=False)
# id_unshifted_11c_test_data, id_unshifted_11c_test_labels = get_flattened_data(id_unshifted_11c_test_dataset)
# id_unshifted_denoised_11c_test_data, id_unshifted_denoised_11c_test_labels = get_flattened_data(id_unshifted_11c_test_dataset, denoise=True, denoise_path=single_chan_ckpt)

True spikes_train.npy
(2000, 21, 121)


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2000/2000 [00:00<00:00, 2108.69it/s]


(2000, 21, 121)


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2000/2000 [00:00<00:00, 2069.38it/s]


True spikes_train.npy
(2000, 21, 121)


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2000/2000 [00:02<00:00, 962.60it/s]


(2000, 21, 121)


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2000/2000 [00:01<00:00, 1002.73it/s]


In [86]:
def pca_gmm_average_tr(train_dataset, memory_dataset, test_dataset, test_labels, num_classes=10, num_iters=50, out_dim=5):
    scores = []
    np.random.seed(0)
    random.seed(0)
    reps_train, _, pca_ = pca(train_dataset, out_dim)
    reps_memory = pca_.transform(memory_dataset)
    reps_test = pca_.transform(test_dataset)
    for i in range(num_iters):
        # covariance_type : {'full', 'tied', 'diag', 'spherical'}
        covariance_type = 'full'
        gmm = GaussianMixture(num_classes,
                            random_state=random.randint(0, 1000000),
                            covariance_type=covariance_type).fit(reps_memory)
        gmm_cont_test_labels = gmm.predict(reps_test)
        curr_score = adjusted_rand_score(test_labels, gmm_cont_test_labels)*100
        scores.append(curr_score)
        if i == 49:
           print("max gmm score: {}".format(max(scores)))
           print("min gmm score: {}".format(min(scores)))
           print("50 run gmm mean score: {}".format(np.mean(scores)))
           print("50 run gmm std-dev score: {}".format(np.std(scores)))

    score = np.mean(scores)

    return score

def pca_gmm_average(memory_dataset, test_dataset, test_labels, num_classes=10, num_iters=50, out_dim=5):
    scores = []
    np.random.seed(0)
    random.seed(0)
    for i in range(num_iters):
        # covariance_type : {'full', 'tied', 'diag', 'spherical'}
        covariance_type = 'full'
        reps_train, _, pca_ = pca(memory_dataset, out_dim)
        reps_test = pca_.transform(test_dataset)
        gmm = GaussianMixture(num_classes,
                            random_state=random.randint(0, 1000000),
                            covariance_type=covariance_type).fit(reps_train)
        gmm_cont_test_labels = gmm.predict(reps_test)
        curr_score = adjusted_rand_score(test_labels, gmm_cont_test_labels)*100
        scores.append(curr_score)
        if i == 49:
           print("max gmm score: {}".format(max(scores)))
           print("min gmm score: {}".format(min(scores)))
           print("50 run gmm mean score: {}".format(np.mean(scores)))
           print("50 run gmm std-dev score: {}".format(np.std(scores)))

    score = np.mean(scores)

    return score

# New Neuron metrics

## 10 neuron, 1200 spikes, 5 channels

In [16]:
dy016_ood_score = pca_gmm_average_tr(dy016_denoised_id_5c_train_data, dy016_denoised_ood_5c_memory_data, dy016_denoised_ood_5c_test_data, dy016_denoised_ood_5c_test_labels)
print("DY016 OOD (1200 train spikes) denoised data GMM score: " + str(dy016_ood_score))
dy009_ood_score = pca_gmm_average_tr(dy016_denoised_id_5c_train_data, dy009_denoised_ood_5c_memory_data, dy009_denoised_ood_5c_test_data, dy009_denoised_ood_5c_test_labels)
print("DY009 OOD (1200 train spikes) denoised data GMM score: " + str(dy009_ood_score))
testset_A_score = pca_gmm_average_tr(dy016_denoised_id_5c_train_data, id_denoised_5c_memory_data, id_denoised_5c_test_data, id_denoised_5c_test_labels)
print("test set A (DY016 ID) (1200 train spikes) ID denoised data GMM score: " + str(testset_A_score))

max gmm score: 49.94842592385938
min gmm score: 42.4561070568621
50 run gmm mean score: 46.41453202384358
50 run gmm std-dev score: 1.641824642630016
DY016 ID (200 train spikes) denoised data GMM score: 46.41453202384358
max gmm score: 43.23980993019478
min gmm score: 36.045935604862564
50 run gmm mean score: 38.97439442092881
50 run gmm std-dev score: 1.319551055255417
DY009 ID (200 train spikes) denoised data GMM score: 38.97439442092881
max gmm score: 55.71084489678194
min gmm score: 33.0873169073262
50 run gmm mean score: 46.00936104828743
50 run gmm std-dev score: 6.792019176866088
test set A (DY016) (200 train spikes) ID denoised data GMM score: 46.00936104828743


In [18]:
dy016_ood_score = pca_gmm_average_tr(dy016_id_5c_train_data, dy016_ood_5c_memory_data, dy016_ood_5c_test_data, dy016_ood_5c_test_labels)
print("DY016 OOD (10 neurons, 1200 train spikes) data GMM score: " + str(dy016_ood_score))
dy009_ood_score = pca_gmm_average_tr(dy016_id_5c_train_data, dy009_ood_5c_memory_data, dy009_ood_5c_test_data, dy009_ood_5c_test_labels)
print("DY009 OOD (10 neurons, 1200 train spikes) data GMM score: " + str(dy009_ood_score))
testset_A_score = pca_gmm_average_tr(dy016_id_5c_train_data, id_5c_memory_data, id_5c_test_data, id_5c_test_labels)
print("test set A (ID DY016) (10 neurons, 1200 train spikes) ID data GMM score: " + str(testset_A_score))

max gmm score: 40.260985988403114
min gmm score: 29.225243863254796
50 run gmm mean score: 34.60790511126595
50 run gmm std-dev score: 2.54071928339712
DY016 OOD (10 neurons, 1200 train spikes) data GMM score: 34.60790511126595
max gmm score: 26.317805234862252
min gmm score: 18.433767303204288
50 run gmm mean score: 21.828724358383383
50 run gmm std-dev score: 2.094112767631093
DY009 OOD (10 neurons, 1200 train spikes) data GMM score: 21.828724358383383
max gmm score: 51.152363011948665
min gmm score: 32.68867536638837
50 run gmm mean score: 43.49786698028805
50 run gmm std-dev score: 5.243246579351943
test set A (ID DY016) (10 neurons, 1200 train spikes) ID data GMM score: 43.49786698028805


## 10 neuron, 1200 spikes, 11 channels

In [38]:
# dy016_ood_score = pca_gmm_average_tr(dy016_denoised_id_11c_train_data, dy016_denoised_ood_11c_memory_data, dy016_denoised_ood_11c_test_data, dy016_denoised_ood_11c_test_labels)
# print("DY016 OOD (1200 train spikes) denoised data GMM score: " + str(dy016_ood_score))
# dy009_ood_score = pca_gmm_average_tr(dy016_denoised_id_11c_train_data, dy009_denoised_ood_11c_memory_data, dy009_denoised_ood_11c_test_data, dy009_denoised_ood_11c_test_labels)
# print("DY009 OOD (1200 train spikes) denoised data GMM score: " + str(dy009_ood_score))
testset_A_score = pca_gmm_average_tr(dy016_denoised_id_11c_train_data, id_denoised_11c_memory_data, id_denoised_11c_test_data, id_denoised_11c_test_labels)
print("test set A (DY016 ID) (1200 train spikes) ID denoised data GMM score: " + str(testset_A_score))

max gmm score: 45.11223772731075
min gmm score: 30.941207470431127
50 run gmm mean score: 38.53493192043495
50 run gmm std-dev score: 4.803947540894395
test set A (DY016 ID) (1200 train spikes) ID denoised data GMM score: 38.53493192043495


In [39]:
# dy016_ood_score = pca_gmm_average_tr(dy016_id_11c_train_data, dy016_ood_11c_memory_data, dy016_ood_11c_test_data, dy016_ood_11c_test_labels)
# print("DY016 OOD (10 neurons, 1200 train spikes) data GMM score: " + str(dy016_ood_score))
# dy009_ood_score = pca_gmm_average_tr(dy016_id_11c_train_data, dy009_ood_11c_memory_data, dy009_ood_11c_test_data, dy009_ood_11c_test_labels)
# print("DY009 OOD (10 neurons, 1200 train spikes) data GMM score: " + str(dy009_ood_score))
testset_A_score = pca_gmm_average_tr(dy016_id_11c_train_data, id_11c_memory_data, id_11c_test_data, id_11c_test_labels)
print("test set A (ID DY016) (10 neurons, 1200 train spikes) ID data GMM score: " + str(testset_A_score))

max gmm score: 48.424282139063166
min gmm score: 32.454088162303144
50 run gmm mean score: 42.3763257936804
50 run gmm std-dev score: 4.32258951624318
test set A (ID DY016) (10 neurons, 1200 train spikes) ID data GMM score: 42.3763257936804


# ZFM OOD recording

## 400 neuron, 1200 spikes, 5 channels

In [87]:
testset_zfm_score = pca_gmm_average_tr(zfm_denoised_5c_memory_data, zfm_denoised_5c_memory_data, zfm_denoised_5c_test_data, zfm_denoised_5c_test_labels)
print("test set ZFM (new recording OOD) (200 train spikes) denoised data GMM score: " + str(testset_zfm_score))

max gmm score: 65.89325561532141
min gmm score: 51.84333377654061
50 run gmm mean score: 57.8391398126883
50 run gmm std-dev score: 4.1142152417722
test set ZFM (new recording OOD) (200 train spikes) denoised data GMM score: 57.8391398126883


In [88]:
testset_zfm_score = pca_gmm_average_tr(zfm_5c_memory_data, zfm_5c_memory_data, zfm_5c_test_data, zfm_5c_test_labels)
print("test set ZFM (new recording OOD) (200 train spikes) data GMM score: " + str(testset_zfm_score))

max gmm score: 63.35312581416943
min gmm score: 47.23009981959982
50 run gmm mean score: 57.921201624894664
50 run gmm std-dev score: 4.049843708782929
test set ZFM (new recording OOD) (200 train spikes) data GMM score: 57.921201624894664


## 400 neuron, 1200 spikes, 11 channels

In [79]:
testset_zfm_score = pca_gmm_average_tr(fourhund_denoised_11c_train_data, zfm_denoised_11c_memory_data, zfm_denoised_11c_test_data, zfm_denoised_11c_test_labels)
print("test set ZFM (new recording OOD) (200 train spikes) denoised data GMM score: " + str(testset_zfm_score))

max gmm score: 47.13053747814232
min gmm score: 33.71500606087366
50 run gmm mean score: 43.450344153478554
50 run gmm std-dev score: 3.178009011823364
test set ZFM (new recording OOD) (200 train spikes) denoised data GMM score: 43.450344153478554


In [80]:
testset_zfm_score = pca_gmm_average_tr(fourhund_11c_train_data, zfm_11c_memory_data, zfm_11c_test_data, zfm_11c_test_labels)
print("test set ZFM (new recording OOD) (200 train spikes) data GMM score: " + str(testset_zfm_score))

max gmm score: 23.79046965113549
min gmm score: 17.421474166473846
50 run gmm mean score: 19.79304988245128
50 run gmm std-dev score: 1.4824715818052951
test set ZFM (new recording OOD) (200 train spikes) data GMM score: 19.79304988245128


In [ ]:
ceed_5chan = [0.66, .69, .72, ]
depca_5chan = [.54, .58, .51, ]
pca_5chan = [.26, .31, .53, ]
ceed_11chan = [0.61, .76, .70, ]
depca_11chan = [.52, .50, .53, ]
pca_11chan = [.28, .30, .50, ]

# UNSHIFTED DATA

## 5D

## 10 neuron, 1200 spikes, 3 channels

In [9]:
# dy016_ood_score = pca_gmm_average_tr(dy016_unshifted_denoised_id_5c_train_data, dy016_denoised_ood_5c_memory_data, dy016_denoised_ood_5c_test_data, dy016_denoised_ood_5c_test_labels)
# print("DY016 OOD (1200 train spikes) denoised data GMM score: " + str(dy016_ood_score))
# dy009_ood_score = pca_gmm_average_tr(dy016_denoised_id_5c_train_data, dy009_denoised_ood_5c_memory_data, dy009_denoised_ood_5c_test_data, dy009_denoised_ood_5c_test_labels)
# print("DY009 OOD (1200 train spikes) denoised data GMM score: " + str(dy009_ood_score))
testset_A_score = pca_gmm_average_tr(dy016_unshifted_denoised_id_3c_train_data, id_unshifted_denoised_3c_memory_data, id_unshifted_denoised_3c_test_data, id_unshifted_denoised_3c_test_labels)
print("test set A (DY016 ID) (1200 train spikes) unshifted ID denoised data GMM score: " + str(testset_A_score))

max gmm score: 76.30853852749749
min gmm score: 45.993274753132226
50 run gmm mean score: 66.35200224815352
50 run gmm std-dev score: 6.030691451046885
test set A (DY016 ID) (1200 train spikes) unshifted ID denoised data GMM score: 66.35200224815352


In [78]:
# dy016_ood_score = pca_gmm_average_tr(dy016_id_5c_train_data, dy016_ood_5c_memory_data, dy016_ood_5c_test_data, dy016_ood_5c_test_labels)
# print("DY016 OOD (10 neurons, 1200 train spikes) data GMM score: " + str(dy016_ood_score))
# dy009_ood_score = pca_gmm_average_tr(dy016_id_5c_train_data, dy009_ood_5c_memory_data, dy009_ood_5c_test_data, dy009_ood_5c_test_labels)
# print("DY009 OOD (10 neurons, 1200 train spikes) data GMM score: " + str(dy009_ood_score))
testset_A_score = pca_gmm_average_tr(dy016_unshifted_id_3c_train_data, id_unshifted_3c_memory_data, id_unshifted_3c_test_data, id_unshifted_3c_test_labels)
print("test set A (ID DY016) (10 neurons, 1200 train spikes) unshifted ID data GMM score: " + str(testset_A_score))

max gmm score: 65.96319758793757
min gmm score: 40.2332860231257
50 run gmm mean score: 60.70185469702851
50 run gmm std-dev score: 3.9961677486698
test set A (ID DY016) (10 neurons, 1200 train spikes) unshifted ID data GMM score: 60.70185469702851


## 10 neuron, 1200 spikes, 5 channels

In [10]:
# dy016_ood_score = pca_gmm_average_tr(dy016_unshifted_denoised_id_5c_train_data, dy016_denoised_ood_5c_memory_data, dy016_denoised_ood_5c_test_data, dy016_denoised_ood_5c_test_labels)
# print("DY016 OOD (1200 train spikes) denoised data GMM score: " + str(dy016_ood_score))
# dy009_ood_score = pca_gmm_average_tr(dy016_denoised_id_5c_train_data, dy009_denoised_ood_5c_memory_data, dy009_denoised_ood_5c_test_data, dy009_denoised_ood_5c_test_labels)
# print("DY009 OOD (1200 train spikes) denoised data GMM score: " + str(dy009_ood_score))
testset_A_score = pca_gmm_average_tr(dy016_unshifted_denoised_id_5c_train_data, id_unshifted_denoised_5c_memory_data, id_unshifted_denoised_5c_test_data, id_unshifted_denoised_5c_test_labels)
print("test set A (DY016 ID) (1200 train spikes) unshifted ID denoised data GMM score: " + str(testset_A_score))

max gmm score: 88.53915483702174
min gmm score: 46.81109537355625
50 run gmm mean score: 70.38432162311973
50 run gmm std-dev score: 6.1667722216090946
test set A (DY016 ID) (1200 train spikes) unshifted ID denoised data GMM score: 70.38432162311973


In [80]:
# dy016_ood_score = pca_gmm_average_tr(dy016_id_5c_train_data, dy016_ood_5c_memory_data, dy016_ood_5c_test_data, dy016_ood_5c_test_labels)
# print("DY016 OOD (10 neurons, 1200 train spikes) data GMM score: " + str(dy016_ood_score))
# dy009_ood_score = pca_gmm_average_tr(dy016_id_5c_train_data, dy009_ood_5c_memory_data, dy009_ood_5c_test_data, dy009_ood_5c_test_labels)
# print("DY009 OOD (10 neurons, 1200 train spikes) data GMM score: " + str(dy009_ood_score))
testset_A_score = pca_gmm_average_tr(dy016_unshifted_id_5c_train_data, id_unshifted_5c_memory_data, id_unshifted_5c_test_data, id_unshifted_5c_test_labels)
print("test set A (ID DY016) (10 neurons, 1200 train spikes) unshifted ID data GMM score: " + str(testset_A_score))

max gmm score: 71.60519361278969
min gmm score: 48.425702189770185
50 run gmm mean score: 66.75133247670489
50 run gmm std-dev score: 3.638737995036119
test set A (ID DY016) (10 neurons, 1200 train spikes) unshifted ID data GMM score: 66.75133247670489


## 10 neuron, 1200 spikes, 11 channels

In [11]:
# dy016_ood_score = pca_gmm_average_tr(dy016_denoised_id_11c_train_data, dy016_denoised_ood_11c_memory_data, dy016_denoised_ood_11c_test_data, dy016_denoised_ood_11c_test_labels)
# print("DY016 OOD (1200 train spikes) denoised data GMM score: " + str(dy016_ood_score))
# dy009_ood_score = pca_gmm_average_tr(dy016_denoised_id_11c_train_data, dy009_denoised_ood_11c_memory_data, dy009_denoised_ood_11c_test_data, dy009_denoised_ood_11c_test_labels)
# print("DY009 OOD (1200 train spikes) denoised data GMM score: " + str(dy009_ood_score))
testset_A_score = pca_gmm_average_tr(dy016_unshifted_denoised_id_11c_train_data, id_denoised_11c_memory_data, id_denoised_11c_test_data, id_denoised_11c_test_labels)
print("test set A (DY016 ID) (1200 train spikes, unshifted) ID denoised data GMM score: " + str(testset_A_score))

max gmm score: 48.38169206354698
min gmm score: 36.982199597751496
50 run gmm mean score: 44.0537510873049
50 run gmm std-dev score: 3.265764618517954
test set A (DY016 ID) (1200 train spikes, unshifted) ID denoised data GMM score: 44.0537510873049


In [82]:
# dy016_ood_score = pca_gmm_average_tr(dy016_id_11c_train_data, dy016_ood_11c_memory_data, dy016_ood_11c_test_data, dy016_ood_11c_test_labels)
# print("DY016 OOD (10 neurons, 1200 train spikes) data GMM score: " + str(dy016_ood_score))
# dy009_ood_score = pca_gmm_average_tr(dy016_id_11c_train_data, dy009_ood_11c_memory_data, dy009_ood_11c_test_data, dy009_ood_11c_test_labels)
# print("DY009 OOD (10 neurons, 1200 train spikes) data GMM score: " + str(dy009_ood_score))
testset_A_score = pca_gmm_average_tr(dy016_unshifted_id_11c_train_data, id_11c_memory_data, id_11c_test_data, id_11c_test_labels)
print("test set A (ID DY016) (10 neurons, 1200 train spikes, unshifted) ID data GMM score: " + str(testset_A_score))

max gmm score: 52.60847520629673
min gmm score: 35.92053452483929
50 run gmm mean score: 45.8206854102799
50 run gmm std-dev score: 4.3143859601744845
test set A (ID DY016) (10 neurons, 1200 train spikes, unshifted) ID data GMM score: 45.8206854102799


## 3D

## 10 neuron, 1200 spikes, 3 channels

In [12]:
dim = 3
# dy016_ood_score = pca_gmm_average_tr(dy016_unshifted_denoised_id_5c_train_data, dy016_denoised_ood_5c_memory_data, dy016_denoised_ood_5c_test_data, dy016_denoised_ood_5c_test_labels)
# print("DY016 OOD (1200 train spikes) denoised data GMM score: " + str(dy016_ood_score))
# dy009_ood_score = pca_gmm_average_tr(dy016_denoised_id_5c_train_data, dy009_denoised_ood_5c_memory_data, dy009_denoised_ood_5c_test_data, dy009_denoised_ood_5c_test_labels)
# print("DY009 OOD (1200 train spikes) denoised data GMM score: " + str(dy009_ood_score))
testset_A_score = pca_gmm_average_tr(dy016_unshifted_denoised_id_3c_train_data, id_unshifted_denoised_3c_memory_data, id_unshifted_denoised_3c_test_data, id_unshifted_denoised_3c_test_labels, out_dim=dim)
print("test set A (DY016 ID) (1200 train spikes) unshifted ID denoised data GMM score: " + str(testset_A_score))

max gmm score: 66.00855102539781
min gmm score: 50.54808865343365
50 run gmm mean score: 60.22607903111482
50 run gmm std-dev score: 3.8696406996201587
test set A (DY016 ID) (1200 train spikes) unshifted ID denoised data GMM score: 60.22607903111482


In [ ]:
# dy016_ood_score = pca_gmm_average_tr(dy016_id_5c_train_data, dy016_ood_5c_memory_data, dy016_ood_5c_test_data, dy016_ood_5c_test_labels)
# print("DY016 OOD (10 neurons, 1200 train spikes) data GMM score: " + str(dy016_ood_score))
# dy009_ood_score = pca_gmm_average_tr(dy016_id_5c_train_data, dy009_ood_5c_memory_data, dy009_ood_5c_test_data, dy009_ood_5c_test_labels)
# print("DY009 OOD (10 neurons, 1200 train spikes) data GMM score: " + str(dy009_ood_score))
testset_A_score = pca_gmm_average_tr(dy016_unshifted_id_3c_train_data, id_unshifted_3c_memory_data, id_unshifted_3c_test_data, id_unshifted_3c_test_labels, out_dim=dim)
print("test set A (ID DY016) (10 neurons, 1200 train spikes) unshifted ID data GMM score: " + str(testset_A_score))

## 10 neuron, 1200 spikes, 5 channels

In [13]:
# dy016_ood_score = pca_gmm_average_tr(dy016_unshifted_denoised_id_5c_train_data, dy016_denoised_ood_5c_memory_data, dy016_denoised_ood_5c_test_data, dy016_denoised_ood_5c_test_labels)
# print("DY016 OOD (1200 train spikes) denoised data GMM score: " + str(dy016_ood_score))
# dy009_ood_score = pca_gmm_average_tr(dy016_denoised_id_5c_train_data, dy009_denoised_ood_5c_memory_data, dy009_denoised_ood_5c_test_data, dy009_denoised_ood_5c_test_labels)
# print("DY009 OOD (1200 train spikes) denoised data GMM score: " + str(dy009_ood_score))
testset_A_score = pca_gmm_average_tr(dy016_unshifted_denoised_id_5c_train_data, id_unshifted_denoised_5c_memory_data, id_unshifted_denoised_5c_test_data, id_unshifted_denoised_5c_test_labels, out_dim=dim)
print("test set A (DY016 ID) (1200 train spikes) unshifted ID denoised data GMM score: " + str(testset_A_score))


max gmm score: 66.53412100798866
min gmm score: 31.18912880747499
50 run gmm mean score: 54.52864285427125
50 run gmm std-dev score: 7.02508213444077
test set A (DY016 ID) (1200 train spikes) unshifted ID denoised data GMM score: 54.52864285427125


In [ ]:
# dy016_ood_score = pca_gmm_average_tr(dy016_id_5c_train_data, dy016_ood_5c_memory_data, dy016_ood_5c_test_data, dy016_ood_5c_test_labels)
# print("DY016 OOD (10 neurons, 1200 train spikes) data GMM score: " + str(dy016_ood_score))
# dy009_ood_score = pca_gmm_average_tr(dy016_id_5c_train_data, dy009_ood_5c_memory_data, dy009_ood_5c_test_data, dy009_ood_5c_test_labels)
# print("DY009 OOD (10 neurons, 1200 train spikes) data GMM score: " + str(dy009_ood_score))
testset_A_score = pca_gmm_average_tr(dy016_unshifted_id_5c_train_data, id_unshifted_5c_memory_data, id_unshifted_5c_test_data, id_unshifted_5c_test_labels, out_dim=dim)
print("test set A (ID DY016) (10 neurons, 1200 train spikes) unshifted ID data GMM score: " + str(testset_A_score))


## 10 neuron, 1200 spikes, 11 channels

In [14]:
# dy016_ood_score = pca_gmm_average_tr(dy016_denoised_id_11c_train_data, dy016_denoised_ood_11c_memory_data, dy016_denoised_ood_11c_test_data, dy016_denoised_ood_11c_test_labels)
# print("DY016 OOD (1200 train spikes) denoised data GMM score: " + str(dy016_ood_score))
# dy009_ood_score = pca_gmm_average_tr(dy016_denoised_id_11c_train_data, dy009_denoised_ood_11c_memory_data, dy009_denoised_ood_11c_test_data, dy009_denoised_ood_11c_test_labels)
# print("DY009 OOD (1200 train spikes) denoised data GMM score: " + str(dy009_ood_score))
testset_A_score = pca_gmm_average_tr(dy016_unshifted_denoised_id_11c_train_data, id_denoised_11c_memory_data, id_denoised_11c_test_data, id_denoised_11c_test_labels, out_dim=dim)
print("test set A (DY016 ID) (1200 train spikes, unshifted) ID denoised data GMM score: " + str(testset_A_score))

max gmm score: 41.36333362566773
min gmm score: 25.609027662111515
50 run gmm mean score: 39.00482181099553
50 run gmm std-dev score: 2.011718397307804
test set A (DY016 ID) (1200 train spikes, unshifted) ID denoised data GMM score: 39.00482181099553


In [ ]:
# dy016_ood_score = pca_gmm_average_tr(dy016_id_11c_train_data, dy016_ood_11c_memory_data, dy016_ood_11c_test_data, dy016_ood_11c_test_labels)
# print("DY016 OOD (10 neurons, 1200 train spikes) data GMM score: " + str(dy016_ood_score))
# dy009_ood_score = pca_gmm_average_tr(dy016_id_11c_train_data, dy009_ood_11c_memory_data, dy009_ood_11c_test_data, dy009_ood_11c_test_labels)
# print("DY009 OOD (10 neurons, 1200 train spikes) data GMM score: " + str(dy009_ood_score))
testset_A_score = pca_gmm_average_tr(dy016_unshifted_id_11c_train_data, id_11c_memory_data, id_11c_test_data, id_11c_test_labels, out_dim=dim)
print("test set A (ID DY016) (10 neurons, 1200 train spikes, unshifted) ID data GMM score: " + str(testset_A_score))

## 2D

## 10 neuron, 1200 spikes, 3 channels

In [15]:
dim = 2
# dy016_ood_score = pca_gmm_average_tr(dy016_unshifted_denoised_id_5c_train_data, dy016_denoised_ood_5c_memory_data, dy016_denoised_ood_5c_test_data, dy016_denoised_ood_5c_test_labels)
# print("DY016 OOD (1200 train spikes) denoised data GMM score: " + str(dy016_ood_score))
# dy009_ood_score = pca_gmm_average_tr(dy016_denoised_id_5c_train_data, dy009_denoised_ood_5c_memory_data, dy009_denoised_ood_5c_test_data, dy009_denoised_ood_5c_test_labels)
# print("DY009 OOD (1200 train spikes) denoised data GMM score: " + str(dy009_ood_score))
testset_A_score = pca_gmm_average_tr(dy016_unshifted_denoised_id_3c_train_data, id_unshifted_denoised_3c_memory_data, id_unshifted_denoised_3c_test_data, id_unshifted_denoised_3c_test_labels, out_dim=dim)
print("test set A (DY016 ID) (1200 train spikes) unshifted ID denoised data GMM score: " + str(testset_A_score))

max gmm score: 48.941234082839266
min gmm score: 44.8093877073006
50 run gmm mean score: 46.964517768178375
50 run gmm std-dev score: 1.239935605228106
test set A (DY016 ID) (1200 train spikes) unshifted ID denoised data GMM score: 46.964517768178375


In [ ]:
# dy016_ood_score = pca_gmm_average_tr(dy016_id_5c_train_data, dy016_ood_5c_memory_data, dy016_ood_5c_test_data, dy016_ood_5c_test_labels)
# print("DY016 OOD (10 neurons, 1200 train spikes) data GMM score: " + str(dy016_ood_score))
# dy009_ood_score = pca_gmm_average_tr(dy016_id_5c_train_data, dy009_ood_5c_memory_data, dy009_ood_5c_test_data, dy009_ood_5c_test_labels)
# print("DY009 OOD (10 neurons, 1200 train spikes) data GMM score: " + str(dy009_ood_score))
testset_A_score = pca_gmm_average_tr(dy016_unshifted_id_3c_train_data, id_unshifted_3c_memory_data, id_unshifted_3c_test_data, id_unshifted_3c_test_labels, out_dim=dim)
print("test set A (ID DY016) (10 neurons, 1200 train spikes) unshifted ID data GMM score: " + str(testset_A_score))

## 10 neuron, 1200 spikes, 5 channels

In [16]:
# dy016_ood_score = pca_gmm_average_tr(dy016_unshifted_denoised_id_5c_train_data, dy016_denoised_ood_5c_memory_data, dy016_denoised_ood_5c_test_data, dy016_denoised_ood_5c_test_labels)
# print("DY016 OOD (1200 train spikes) denoised data GMM score: " + str(dy016_ood_score))
# dy009_ood_score = pca_gmm_average_tr(dy016_denoised_id_5c_train_data, dy009_denoised_ood_5c_memory_data, dy009_denoised_ood_5c_test_data, dy009_denoised_ood_5c_test_labels)
# print("DY009 OOD (1200 train spikes) denoised data GMM score: " + str(dy009_ood_score))
testset_A_score = pca_gmm_average_tr(dy016_unshifted_denoised_id_5c_train_data, id_unshifted_denoised_5c_memory_data, id_unshifted_denoised_5c_test_data, id_unshifted_denoised_5c_test_labels, out_dim=dim)
print("test set A (DY016 ID) (1200 train spikes) unshifted ID denoised data GMM score: " + str(testset_A_score))


max gmm score: 51.39686680519359
min gmm score: 46.562615264111116
50 run gmm mean score: 48.98216236033746
50 run gmm std-dev score: 1.1805603676421002
test set A (DY016 ID) (1200 train spikes) unshifted ID denoised data GMM score: 48.98216236033746


In [ ]:
# dy016_ood_score = pca_gmm_average_tr(dy016_id_5c_train_data, dy016_ood_5c_memory_data, dy016_ood_5c_test_data, dy016_ood_5c_test_labels)
# print("DY016 OOD (10 neurons, 1200 train spikes) data GMM score: " + str(dy016_ood_score))
# dy009_ood_score = pca_gmm_average_tr(dy016_id_5c_train_data, dy009_ood_5c_memory_data, dy009_ood_5c_test_data, dy009_ood_5c_test_labels)
# print("DY009 OOD (10 neurons, 1200 train spikes) data GMM score: " + str(dy009_ood_score))
testset_A_score = pca_gmm_average_tr(dy016_unshifted_id_5c_train_data, id_unshifted_5c_memory_data, id_unshifted_5c_test_data, id_unshifted_5c_test_labels, out_dim=dim)
print("test set A (ID DY016) (10 neurons, 1200 train spikes) unshifted ID data GMM score: " + str(testset_A_score))


## 10 neuron, 1200 spikes, 11 channels

In [17]:
# dy016_ood_score = pca_gmm_average_tr(dy016_denoised_id_11c_train_data, dy016_denoised_ood_11c_memory_data, dy016_denoised_ood_11c_test_data, dy016_denoised_ood_11c_test_labels)
# print("DY016 OOD (1200 train spikes) denoised data GMM score: " + str(dy016_ood_score))
# dy009_ood_score = pca_gmm_average_tr(dy016_denoised_id_11c_train_data, dy009_denoised_ood_11c_memory_data, dy009_denoised_ood_11c_test_data, dy009_denoised_ood_11c_test_labels)
# print("DY009 OOD (1200 train spikes) denoised data GMM score: " + str(dy009_ood_score))
testset_A_score = pca_gmm_average_tr(dy016_unshifted_denoised_id_11c_train_data, id_denoised_11c_memory_data, id_denoised_11c_test_data, id_denoised_11c_test_labels, out_dim=dim)
print("test set A (DY016 ID) (1200 train spikes, unshifted) ID denoised data GMM score: " + str(testset_A_score))

max gmm score: 46.26000710486856
min gmm score: 40.35075927008782
50 run gmm mean score: 42.72871192107223
50 run gmm std-dev score: 1.3126639974493244
test set A (DY016 ID) (1200 train spikes, unshifted) ID denoised data GMM score: 42.72871192107223


In [ ]:
# dy016_ood_score = pca_gmm_average_tr(dy016_id_11c_train_data, dy016_ood_11c_memory_data, dy016_ood_11c_test_data, dy016_ood_11c_test_labels)
# print("DY016 OOD (10 neurons, 1200 train spikes) data GMM score: " + str(dy016_ood_score))
# dy009_ood_score = pca_gmm_average_tr(dy016_id_11c_train_data, dy009_ood_11c_memory_data, dy009_ood_11c_test_data, dy009_ood_11c_test_labels)
# print("DY009 OOD (10 neurons, 1200 train spikes) data GMM score: " + str(dy009_ood_score))
testset_A_score = pca_gmm_average_tr(dy016_unshifted_id_11c_train_data, id_11c_memory_data, id_11c_test_data, id_11c_test_labels, out_dim=dim)
print("test set A (ID DY016) (10 neurons, 1200 train spikes, unshifted) ID data GMM score: " + str(testset_A_score))

## 10D

## 10 neuron, 1200 spikes, 3 channels

In [18]:
dim = 10
# dy016_ood_score = pca_gmm_average_tr(dy016_unshifted_denoised_id_5c_train_data, dy016_denoised_ood_5c_memory_data, dy016_denoised_ood_5c_test_data, dy016_denoised_ood_5c_test_labels)
# print("DY016 OOD (1200 train spikes) denoised data GMM score: " + str(dy016_ood_score))
# dy009_ood_score = pca_gmm_average_tr(dy016_denoised_id_5c_train_data, dy009_denoised_ood_5c_memory_data, dy009_denoised_ood_5c_test_data, dy009_denoised_ood_5c_test_labels)
# print("DY009 OOD (1200 train spikes) denoised data GMM score: " + str(dy009_ood_score))
testset_A_score = pca_gmm_average_tr(dy016_unshifted_denoised_id_3c_train_data, id_unshifted_denoised_3c_memory_data, id_unshifted_denoised_3c_test_data, id_unshifted_denoised_3c_test_labels, out_dim=dim)
print("test set A (DY016 ID) (1200 train spikes) unshifted ID denoised data GMM score: " + str(testset_A_score))

max gmm score: 82.6561392404478
min gmm score: 58.301906193863104
50 run gmm mean score: 72.77005206267118
50 run gmm std-dev score: 4.9595837892275085
test set A (DY016 ID) (1200 train spikes) unshifted ID denoised data GMM score: 72.77005206267118


In [ ]:
# dy016_ood_score = pca_gmm_average_tr(dy016_id_5c_train_data, dy016_ood_5c_memory_data, dy016_ood_5c_test_data, dy016_ood_5c_test_labels)
# print("DY016 OOD (10 neurons, 1200 train spikes) data GMM score: " + str(dy016_ood_score))
# dy009_ood_score = pca_gmm_average_tr(dy016_id_5c_train_data, dy009_ood_5c_memory_data, dy009_ood_5c_test_data, dy009_ood_5c_test_labels)
# print("DY009 OOD (10 neurons, 1200 train spikes) data GMM score: " + str(dy009_ood_score))
testset_A_score = pca_gmm_average_tr(dy016_unshifted_id_3c_train_data, id_unshifted_3c_memory_data, id_unshifted_3c_test_data, id_unshifted_3c_test_labels, out_dim=dim)
print("test set A (ID DY016) (10 neurons, 1200 train spikes) unshifted ID data GMM score: " + str(testset_A_score))

## 10 neuron, 1200 spikes, 5 channels

In [19]:
dim = 10
# dy016_ood_score = pca_gmm_average_tr(dy016_unshifted_denoised_id_5c_train_data, dy016_denoised_ood_5c_memory_data, dy016_denoised_ood_5c_test_data, dy016_denoised_ood_5c_test_labels)
# print("DY016 OOD (1200 train spikes) denoised data GMM score: " + str(dy016_ood_score))
# dy009_ood_score = pca_gmm_average_tr(dy016_denoised_id_5c_train_data, dy009_denoised_ood_5c_memory_data, dy009_denoised_ood_5c_test_data, dy009_denoised_ood_5c_test_labels)
# print("DY009 OOD (1200 train spikes) denoised data GMM score: " + str(dy009_ood_score))
testset_A_score = pca_gmm_average_tr(dy016_unshifted_denoised_id_5c_train_data, id_unshifted_denoised_5c_memory_data, id_unshifted_denoised_5c_test_data, id_unshifted_denoised_5c_test_labels, out_dim=dim)
print("test set A (DY016 ID) (1200 train spikes) unshifted ID denoised data GMM score: " + str(testset_A_score))


max gmm score: 95.59290347984087
min gmm score: 59.700709968268725
50 run gmm mean score: 81.93192996065864
50 run gmm std-dev score: 8.064676916804142
test set A (DY016 ID) (1200 train spikes) unshifted ID denoised data GMM score: 81.93192996065864


In [ ]:
# dy016_ood_score = pca_gmm_average_tr(dy016_id_5c_train_data, dy016_ood_5c_memory_data, dy016_ood_5c_test_data, dy016_ood_5c_test_labels)
# print("DY016 OOD (10 neurons, 1200 train spikes) data GMM score: " + str(dy016_ood_score))
# dy009_ood_score = pca_gmm_average_tr(dy016_id_5c_train_data, dy009_ood_5c_memory_data, dy009_ood_5c_test_data, dy009_ood_5c_test_labels)
# print("DY009 OOD (10 neurons, 1200 train spikes) data GMM score: " + str(dy009_ood_score))
testset_A_score = pca_gmm_average_tr(dy016_unshifted_id_5c_train_data, id_unshifted_5c_memory_data, id_unshifted_5c_test_data, id_unshifted_5c_test_labels, out_dim=dim)
print("test set A (ID DY016) (10 neurons, 1200 train spikes) unshifted ID data GMM score: " + str(testset_A_score))


## 10 neuron, 1200 spikes, 11 channels

In [20]:
# dy016_ood_score = pca_gmm_average_tr(dy016_denoised_id_11c_train_data, dy016_denoised_ood_11c_memory_data, dy016_denoised_ood_11c_test_data, dy016_denoised_ood_11c_test_labels)
# print("DY016 OOD (1200 train spikes) denoised data GMM score: " + str(dy016_ood_score))
# dy009_ood_score = pca_gmm_average_tr(dy016_denoised_id_11c_train_data, dy009_denoised_ood_11c_memory_data, dy009_denoised_ood_11c_test_data, dy009_denoised_ood_11c_test_labels)
# print("DY009 OOD (1200 train spikes) denoised data GMM score: " + str(dy009_ood_score))
testset_A_score = pca_gmm_average_tr(dy016_unshifted_denoised_id_11c_train_data, id_denoised_11c_memory_data, id_denoised_11c_test_data, id_denoised_11c_test_labels, out_dim=dim)
print("test set A (DY016 ID) (1200 train spikes, unshifted) ID denoised data GMM score: " + str(testset_A_score))

max gmm score: 59.07042922868088
min gmm score: 44.40798976749304
50 run gmm mean score: 53.83105451524947
50 run gmm std-dev score: 3.410861492467726
test set A (DY016 ID) (1200 train spikes, unshifted) ID denoised data GMM score: 53.83105451524947


In [ ]:
# dy016_ood_score = pca_gmm_average_tr(dy016_id_11c_train_data, dy016_ood_11c_memory_data, dy016_ood_11c_test_data, dy016_ood_11c_test_labels)
# print("DY016 OOD (10 neurons, 1200 train spikes) data GMM score: " + str(dy016_ood_score))
# dy009_ood_score = pca_gmm_average_tr(dy016_id_11c_train_data, dy009_ood_11c_memory_data, dy009_ood_11c_test_data, dy009_ood_11c_test_labels)
# print("DY009 OOD (10 neurons, 1200 train spikes) data GMM score: " + str(dy009_ood_score))
testset_A_score = pca_gmm_average_tr(dy016_unshifted_id_11c_train_data, id_11c_memory_data, id_11c_test_data, id_11c_test_labels, out_dim=dim)
print("test set A (ID DY016) (10 neurons, 1200 train spikes, unshifted) ID data GMM score: " + str(testset_A_score))

# END UNSHIFTED TESTS

## 10 neuron, 200 spikes, 11 channels

In [40]:
# dy016_ood_score = pca_gmm_average_tr(id_denoised_11c_memory_data, dy016_denoised_ood_11c_memory_data, dy016_denoised_ood_11c_test_data, dy016_denoised_ood_11c_test_labels)
# print("DY016 OOD (200 train spikes) denoised data GMM score: " + str(dy016_ood_score))
# dy009_ood_score = pca_gmm_average_tr(id_denoised_11c_memory_data, dy009_denoised_ood_11c_memory_data, dy009_denoised_ood_11c_test_data, dy009_denoised_ood_11c_test_labels)
# print("DY009 OOD (200 train spikes) denoised data GMM score: " + str(dy009_ood_score))
testset_A_score = pca_gmm_average_tr(id_denoised_11c_memory_data, id_denoised_11c_memory_data, id_denoised_11c_test_data, id_denoised_11c_test_labels)
print("test set A (DY016 ID) (200 train spikes) ID denoised data GMM score: " + str(testset_A_score))

max gmm score: 41.11059022606896
min gmm score: 29.91931955444329
50 run gmm mean score: 39.08140017287369
50 run gmm std-dev score: 2.8374318290906606
test set A (DY016 ID) (200 train spikes) ID denoised data GMM score: 39.08140017287369


In [41]:
# dy016_ood_score = pca_gmm_average_tr(id_11c_memory_data, dy016_ood_11c_memory_data, dy016_ood_11c_test_data, dy016_ood_11c_test_labels)
# print("DY016 OOD (10 neurons, 200 train spikes) data GMM score: " + str(dy016_ood_score))
# dy009_ood_score = pca_gmm_average_tr(id_11c_memory_data, dy009_ood_11c_memory_data, dy009_ood_11c_test_data, dy009_ood_11c_test_labels)
# print("DY009 OOD (10 neurons, 200 train spikes) data GMM score: " + str(dy009_ood_score))
testset_A_score = pca_gmm_average_tr(id_11c_memory_data, id_11c_memory_data, id_11c_test_data, id_11c_test_labels)
print("test set A (ID DY016) (10 neurons, 200 train spikes) ID data GMM score: " + str(testset_A_score))

max gmm score: 48.1801530981552
min gmm score: 33.90870090124835
50 run gmm mean score: 40.99033602493144
50 run gmm std-dev score: 3.4998472420404356
test set A (ID DY016) (10 neurons, 200 train spikes) ID data GMM score: 40.99033602493144


## 400 neuron, 200 spikes, 5 chan

In [24]:
dy016_ood_score = pca_gmm_average_tr(fourhund_denoised_5c_train_data, dy016_denoised_ood_5c_memory_data, dy016_denoised_ood_5c_test_data, dy016_denoised_ood_5c_test_labels)
print("DY016 OOD (400 neurons, 200 train spikes) denoised data GMM score: " + str(dy016_ood_score))
dy009_ood_score = pca_gmm_average_tr(fourhund_denoised_5c_train_data, dy009_denoised_ood_5c_memory_data, dy009_denoised_ood_5c_test_data, dy009_denoised_ood_5c_test_labels)
print("DY009 OOD (400 neurons, 200 train spikes) denoised data GMM score: " + str(dy009_ood_score))
testset_A_score = pca_gmm_average_tr(fourhund_denoised_5c_train_data, id_denoised_5c_memory_data, id_denoised_5c_test_data, id_denoised_5c_test_labels)
print("test set A (ID DY016) (400 neurons, 200 train spikes) ID denoised data GMM score: " + str(testset_A_score))

max gmm score: 48.852660255138034
min gmm score: 42.61961320461751
50 run gmm mean score: 45.859394513078385
50 run gmm std-dev score: 1.6699787348189143
DY016 OOD (400 neurons, 200 train spikes) denoised data GMM score: 45.859394513078385
max gmm score: 43.50847543736153
min gmm score: 35.619292892656055
50 run gmm mean score: 39.15385726020336
50 run gmm std-dev score: 2.1699930947155828
DY009 OOD (400 neurons, 200 train spikes) denoised data GMM score: 39.15385726020336
max gmm score: 54.64599017617606
min gmm score: 38.370586128739106
50 run gmm mean score: 49.26934635422893
50 run gmm std-dev score: 4.472748220905369
test set A (ID DY016) (400 neurons, 200 train spikes) ID denoised data GMM score: 49.26934635422893


In [25]:
dy016_ood_score = pca_gmm_average_tr(fourhund_5c_train_data, dy016_ood_5c_memory_data, dy016_ood_5c_test_data, dy016_ood_5c_test_labels)
print("DY016 OOD (400 neurons, 200 train spikes) data GMM score: " + str(dy016_ood_score))
dy009_ood_score = pca_gmm_average_tr(fourhund_5c_train_data, dy009_ood_5c_memory_data, dy009_ood_5c_test_data, dy009_ood_5c_test_labels)
print("DY009 OOD (400 neurons, 200 train spikes) data GMM score: " + str(dy009_ood_score))
testset_A_score = pca_gmm_average_tr(fourhund_5c_train_data, id_5c_memory_data, id_5c_test_data, id_5c_test_labels)
print("test set A (ID DY016) (400 neurons, 200 train spikes) ID data GMM score: " + str(testset_A_score))

max gmm score: 42.5790105809703
min gmm score: 24.839751754514143
50 run gmm mean score: 34.62326199084467
50 run gmm std-dev score: 4.1181057189157615
DY016 OOD (400 neurons, 200 train spikes) data GMM score: 34.62326199084467
max gmm score: 23.64360815251711
min gmm score: 15.598370364436695
50 run gmm mean score: 19.737513415238556
50 run gmm std-dev score: 1.5803748763485617
DY009 OOD (400 neurons, 200 train spikes) denoised data GMM score: 19.737513415238556
max gmm score: 48.41062989522548
min gmm score: 36.001991663148324
50 run gmm mean score: 43.56754906865924
50 run gmm std-dev score: 3.468305734304177
test set A (ID DY016) (400 neurons, 200 train spikes) ID data GMM score: 43.56754906865924


## 400 neuron, 200 spikes, 11 chan

In [26]:
dy016_ood_score = pca_gmm_average_tr(fourhund_denoised_11c_train_data, dy016_denoised_ood_11c_memory_data, dy016_denoised_ood_11c_test_data, dy016_denoised_ood_11c_test_labels)
print("DY016 OOD (400 neurons, 200 train spikes) denoised data GMM score: " + str(dy016_ood_score))
dy009_ood_score = pca_gmm_average_tr(fourhund_denoised_11c_train_data, dy009_denoised_ood_11c_memory_data, dy009_denoised_ood_11c_test_data, dy009_denoised_ood_11c_test_labels)
print("DY009 OOD (400 neurons, 200 train spikes) denoised data GMM score: " + str(dy009_ood_score))
testset_A_score = pca_gmm_average_tr(fourhund_denoised_11c_train_data, id_denoised_11c_memory_data, id_denoised_11c_test_data, id_denoised_11c_test_labels)
print("test set A (ID DY016) (400 neurons, 200 train spikes) ID denoised data GMM score: " + str(testset_A_score))

max gmm score: 48.244958295217174
min gmm score: 40.4848033111489
50 run gmm mean score: 46.448484524812066
50 run gmm std-dev score: 1.4455020400279521
DY016 OOD (400 neurons, 200 train spikes) denoised data GMM score: 46.448484524812066
max gmm score: 45.47727678049991
min gmm score: 38.70774512786305
50 run gmm mean score: 42.776622635266406
50 run gmm std-dev score: 1.8247957739373344
DY009 OOD (400 neurons, 200 train spikes) denoised data GMM score: 42.776622635266406
max gmm score: 54.28607334870959
min gmm score: 34.691665530745844
50 run gmm mean score: 42.675400186749386
50 run gmm std-dev score: 3.2538126642336005
test set A (ID DY016) (400 neurons, 200 train spikes) ID denoised data GMM score: 42.675400186749386


In [27]:
dy016_ood_score = pca_gmm_average_tr(fourhund_11c_train_data, dy016_ood_11c_memory_data, dy016_ood_11c_test_data, dy016_ood_11c_test_labels)
print("DY016 OOD (400 neurons, 200 train spikes) data GMM score: " + str(dy016_ood_score))
dy009_ood_score = pca_gmm_average_tr(fourhund_11c_train_data, dy009_ood_11c_memory_data, dy009_ood_11c_test_data, dy009_ood_11c_test_labels)
print("DY009 OOD (400 neurons, 200 train spikes) denoised data GMM score: " + str(dy009_ood_score))
testset_A_score = pca_gmm_average_tr(fourhund_11c_train_data, id_11c_memory_data, id_11c_test_data, id_11c_test_labels)
print("test set A (ID DY016) (400 neurons, 200 train spikes) ID data GMM score: " + str(testset_A_score))

max gmm score: 41.60675125017051
min gmm score: 30.786828103802954
50 run gmm mean score: 36.10275655248079
50 run gmm std-dev score: 2.521434748232127
DY016 OOD (400 neurons, 200 train spikes) data GMM score: 36.10275655248079
max gmm score: 37.55857981940917
min gmm score: 27.450000211494384
50 run gmm mean score: 31.2089024732494
50 run gmm std-dev score: 2.53233762744236
DY009 OOD (400 neurons, 200 train spikes) denoised data GMM score: 31.2089024732494
max gmm score: 44.705760996409985
min gmm score: 33.909147579717605
50 run gmm mean score: 40.77009849126654
50 run gmm std-dev score: 3.3294820050708007
test set A (ID DY016) (400 neurons, 200 train spikes) ID data GMM score: 40.77009849126654


### 2D

In [24]:
dy016_ood_score = pca_gmm_average_tr(fourhund_denoised_11c_train_data, dy016_denoised_ood_11c_memory_data, dy016_denoised_ood_11c_test_data, dy016_denoised_ood_11c_test_labels, out_dim=2)
print("\nDY016 OOD (390 neurons, 200 train spikes) denoised PCA 2D GMM score: " + str(dy016_ood_score) + "\n")
dy009_ood_score = pca_gmm_average_tr(fourhund_denoised_11c_train_data, dy009_denoised_ood_11c_memory_data, dy009_denoised_ood_11c_test_data, dy009_denoised_ood_11c_test_labels, out_dim=2)
print("\nDY009 OOD (390 neurons, 200 train spikes) denoised PCA 2D GMM score: " + str(dy009_ood_score) + "\n")
testset_A_score = pca_gmm_average_tr(fourhund_denoised_11c_train_data, id_denoised_11c_memory_data, id_denoised_11c_test_data, id_denoised_11c_test_labels, out_dim=2)
print("\ntest set A (real OOD DY016) (390 neurons, 200 train spikes) denoised PCA 2D GMM score: " + str(testset_A_score) + "\n")

max gmm score: 31.367574020052764
min gmm score: 27.38031033848398
50 run gmm mean score: 29.244440577614792
50 run gmm std-dev score: 1.0886161482527004

DY016 OOD (390 neurons, 200 train spikes) denoised PCA 2D GMM score: 29.244440577614792

max gmm score: 30.870854321855884
min gmm score: 24.50727620625854
50 run gmm mean score: 28.809550748941007
50 run gmm std-dev score: 1.1716856479121662

DY009 OOD (390 neurons, 200 train spikes) denoised PCA 2D GMM score: 28.809550748941007

max gmm score: 36.63381960074184
min gmm score: 26.59223611376787
50 run gmm mean score: 33.520080756522894
50 run gmm std-dev score: 2.4862008372452036

test set A (real OOD DY016) (390 neurons, 200 train spikes) denoised PCA 2D GMM score: 33.520080756522894



In [25]:
dy016_ood_score = pca_gmm_average_tr(fourhund_11c_train_data, dy016_ood_11c_memory_data, dy016_ood_11c_test_data, dy016_ood_11c_test_labels, out_dim=2)
print("DY016 OOD (400 neurons, 200 train spikes) PCA 2D GMM score: " + str(dy016_ood_score) + "\n")
dy009_ood_score = pca_gmm_average_tr(fourhund_11c_train_data, dy009_ood_11c_memory_data, dy009_ood_11c_test_data, dy009_ood_11c_test_labels, out_dim=2)
print("DY009 OOD (400 neurons, 200 train spikes) PCA 2D GMM score: " + str(dy009_ood_score) + "\n")
testset_A_score = pca_gmm_average_tr(fourhund_11c_train_data, id_11c_memory_data, id_11c_test_data, id_11c_test_labels, out_dim=2)
print("test set A (ID DY016) (400 neurons, 200 train spikes) ID PCA 2D GMM score: " + str(testset_A_score) + "\n")

max gmm score: 33.167844396349736
min gmm score: 28.7783684046229
50 run gmm mean score: 31.014090855677026
50 run gmm std-dev score: 1.0324469274778698
DY016 OOD (400 neurons, 200 train spikes) PCA 2D GMM score: 31.014090855677026

max gmm score: 34.05127026658665
min gmm score: 27.391791501622
50 run gmm mean score: 31.74023959848015
50 run gmm std-dev score: 1.4542760976541103
DY009 OOD (400 neurons, 200 train spikes) PCA 2D GMM score: 31.74023959848015

max gmm score: 38.821342632509115
min gmm score: 32.91445827202066
50 run gmm mean score: 36.99547710870561
50 run gmm std-dev score: 1.7696993427193137
test set A (ID DY016) (400 neurons, 200 train spikes) ID PCA 2D GMM score: 36.99547710870561



### 3D

In [26]:
dim = 3
dy016_ood_score = pca_gmm_average_tr(fourhund_denoised_11c_train_data, dy016_denoised_ood_11c_memory_data, dy016_denoised_ood_11c_test_data, dy016_denoised_ood_11c_test_labels, out_dim=dim)
print(f"\nDY016 OOD (390 neurons, 200 train spikes) denoised PCA {dim}D GMM score: " + str(dy016_ood_score) + "\n")
dy009_ood_score = pca_gmm_average_tr(fourhund_denoised_11c_train_data, dy009_denoised_ood_11c_memory_data, dy009_denoised_ood_11c_test_data, dy009_denoised_ood_11c_test_labels, out_dim=dim)
print(f"\nDY009 OOD (390 neurons, 200 train spikes) denoised PCA {dim}D GMM score: " + str(dy009_ood_score) + "\n")
testset_A_score = pca_gmm_average_tr(fourhund_denoised_11c_train_data, id_denoised_11c_memory_data, id_denoised_11c_test_data, id_denoised_11c_test_labels, out_dim=dim)
print(f"\ntest set A (real OOD DY016) (390 neurons, 200 train spikes) denoised PCA {dim}D GMM score: " + str(testset_A_score) + "\n")

max gmm score: 39.01603204195286
min gmm score: 32.67454849350456
50 run gmm mean score: 36.17061203820948
50 run gmm std-dev score: 1.3058295216570994

DY016 OOD (390 neurons, 200 train spikes) denoised PCA 3D GMM score: 36.17061203820948

max gmm score: 36.188735090611175
min gmm score: 27.87475251794593
50 run gmm mean score: 33.70920759211706
50 run gmm std-dev score: 1.8005758078270524

DY009 OOD (390 neurons, 200 train spikes) denoised PCA 3D GMM score: 33.70920759211706

max gmm score: 44.27892692399163
min gmm score: 36.37786716203638
50 run gmm mean score: 40.57326114160049
50 run gmm std-dev score: 1.5958209221026214

test set A (real OOD DY016) (390 neurons, 200 train spikes) denoised PCA 3D GMM score: 40.57326114160049



In [27]:
dim = 3
dy016_ood_score = pca_gmm_average_tr(fourhund_11c_train_data, dy016_ood_11c_memory_data, dy016_ood_11c_test_data, dy016_ood_11c_test_labels, out_dim=dim)
print(f"DY016 OOD (400 neurons, 200 train spikes) PCA {dim}D GMM score: " + str(dy016_ood_score) + "\n")
dy009_ood_score = pca_gmm_average_tr(fourhund_11c_train_data, dy009_ood_11c_memory_data, dy009_ood_11c_test_data, dy009_ood_11c_test_labels, out_dim=dim)
print(f"DY009 OOD (400 neurons, 200 train spikes) PCA {dim}D GMM score: " + str(dy009_ood_score) + "\n")
testset_A_score = pca_gmm_average_tr(fourhund_11c_train_data, id_11c_memory_data, id_11c_test_data, id_11c_test_labels, out_dim=dim)
print(f"test set A (ID DY016) (400 neurons, 200 train spikes) ID PCA {dim}D GMM score: " + str(testset_A_score) + "\n")

max gmm score: 39.57835323294429
min gmm score: 28.624295494070175
50 run gmm mean score: 36.137226517439245
50 run gmm std-dev score: 1.9192390064517644
DY016 OOD (400 neurons, 200 train spikes) PCA 3D GMM score: 36.137226517439245

max gmm score: 36.395982959319525
min gmm score: 24.766110069731457
50 run gmm mean score: 31.458855826096627
50 run gmm std-dev score: 2.467101437468211
DY009 OOD (400 neurons, 200 train spikes) PCA 3D GMM score: 31.458855826096627

max gmm score: 42.29058232596206
min gmm score: 31.951975464239023
50 run gmm mean score: 39.588873535457154
50 run gmm std-dev score: 2.0603759007301834
test set A (ID DY016) (400 neurons, 200 train spikes) ID PCA 3D GMM score: 39.588873535457154



### 4D

In [29]:
dim = 4
dy016_ood_score = pca_gmm_average_tr(fourhund_denoised_11c_train_data, dy016_denoised_ood_11c_memory_data, dy016_denoised_ood_11c_test_data, dy016_denoised_ood_11c_test_labels, out_dim=dim)
print(f"\nDY016 OOD (390 neurons, 200 train spikes) denoised PCA {dim}D GMM score: " + str(dy016_ood_score) + "\n")
dy009_ood_score = pca_gmm_average_tr(fourhund_denoised_11c_train_data, dy009_denoised_ood_11c_memory_data, dy009_denoised_ood_11c_test_data, dy009_denoised_ood_11c_test_labels, out_dim=dim)
print(f"\nDY009 OOD (390 neurons, 200 train spikes) denoised PCA {dim}D GMM score: " + str(dy009_ood_score) + "\n")
testset_A_score = pca_gmm_average_tr(fourhund_denoised_11c_train_data, id_denoised_11c_memory_data, id_denoised_11c_test_data, id_denoised_11c_test_labels, out_dim=dim)
print(f"\ntest set A (real OOD DY016) (390 neurons, 200 train spikes) denoised PCA {dim}D GMM score: " + str(testset_A_score) + "\n")

max gmm score: 44.70991818431373
min gmm score: 37.045341394642676
50 run gmm mean score: 41.52627591657378
50 run gmm std-dev score: 1.7388605350810977

DY016 OOD (390 neurons, 200 train spikes) denoised PCA 4D GMM score: 41.52627591657378

max gmm score: 41.62855275072068
min gmm score: 34.971295541103785
50 run gmm mean score: 38.51223397439103
50 run gmm std-dev score: 1.4461434718493789

DY009 OOD (390 neurons, 200 train spikes) denoised PCA 4D GMM score: 38.51223397439103

max gmm score: 43.9988763788631
min gmm score: 30.411199751991447
50 run gmm mean score: 38.69095555443882
50 run gmm std-dev score: 3.773274400859903

test set A (real OOD DY016) (390 neurons, 200 train spikes) denoised PCA 4D GMM score: 38.69095555443882



In [30]:
dim = 4
dy016_ood_score = pca_gmm_average_tr(fourhund_11c_train_data, dy016_ood_11c_memory_data, dy016_ood_11c_test_data, dy016_ood_11c_test_labels, out_dim=dim)
print(f"DY016 OOD (400 neurons, 200 train spikes) PCA {dim}D GMM score: " + str(dy016_ood_score) + "\n")
dy009_ood_score = pca_gmm_average_tr(fourhund_11c_train_data, dy009_ood_11c_memory_data, dy009_ood_11c_test_data, dy009_ood_11c_test_labels, out_dim=dim)
print(f"DY009 OOD (400 neurons, 200 train spikes) PCA {dim}D GMM score: " + str(dy009_ood_score) + "\n")
testset_A_score = pca_gmm_average_tr(fourhund_11c_train_data, id_11c_memory_data, id_11c_test_data, id_11c_test_labels, out_dim=dim)
print(f"test set A (ID DY016) (400 neurons, 200 train spikes) ID PCA {dim}D GMM score: " + str(testset_A_score) + "\n")

max gmm score: 41.79262785828829
min gmm score: 35.171255357500684
50 run gmm mean score: 38.69434611909317
50 run gmm std-dev score: 1.2166306754951735
DY016 OOD (400 neurons, 200 train spikes) PCA 4D GMM score: 38.69434611909317

max gmm score: 39.37748449070801
min gmm score: 26.886333918583148
50 run gmm mean score: 31.851110424729104
50 run gmm std-dev score: 2.575194005705665
DY009 OOD (400 neurons, 200 train spikes) PCA 4D GMM score: 31.851110424729104

max gmm score: 45.056046556912115
min gmm score: 31.127771864885332
50 run gmm mean score: 40.69499622416758
50 run gmm std-dev score: 3.1956207068698204
test set A (ID DY016) (400 neurons, 200 train spikes) ID PCA 4D GMM score: 40.69499622416758



### 6D

In [31]:
dim = 6
dy016_ood_score = pca_gmm_average_tr(fourhund_denoised_11c_train_data, dy016_denoised_ood_11c_memory_data, dy016_denoised_ood_11c_test_data, dy016_denoised_ood_11c_test_labels, out_dim=dim)
print(f"\nDY016 OOD (390 neurons, 200 train spikes) denoised PCA {dim}D GMM score: " + str(dy016_ood_score) + "\n")
dy009_ood_score = pca_gmm_average_tr(fourhund_denoised_11c_train_data, dy009_denoised_ood_11c_memory_data, dy009_denoised_ood_11c_test_data, dy009_denoised_ood_11c_test_labels, out_dim=dim)
print(f"\nDY009 OOD (390 neurons, 200 train spikes) denoised PCA {dim}D GMM score: " + str(dy009_ood_score) + "\n")
testset_A_score = pca_gmm_average_tr(fourhund_denoised_11c_train_data, id_denoised_11c_memory_data, id_denoised_11c_test_data, id_denoised_11c_test_labels, out_dim=dim)
print(f"\ntest set A (real OOD DY016) (390 neurons, 200 train spikes) denoised PCA {dim}D GMM score: " + str(testset_A_score) + "\n")

max gmm score: 53.59458729227635
min gmm score: 44.923582598294786
50 run gmm mean score: 51.05898662925873
50 run gmm std-dev score: 1.8648315210679824

DY016 OOD (390 neurons, 200 train spikes) denoised PCA 6D GMM score: 51.05898662925873

max gmm score: 54.91883163794719
min gmm score: 33.230882664272485
50 run gmm mean score: 46.426277570261185
50 run gmm std-dev score: 3.46394505121059

DY009 OOD (390 neurons, 200 train spikes) denoised PCA 6D GMM score: 46.426277570261185

max gmm score: 51.60619406094356
min gmm score: 39.58927279373619
50 run gmm mean score: 46.60302426691909
50 run gmm std-dev score: 3.013585282046302

test set A (real OOD DY016) (390 neurons, 200 train spikes) denoised PCA 6D GMM score: 46.60302426691909



In [32]:
dim = 6
dy016_ood_score = pca_gmm_average_tr(fourhund_11c_train_data, dy016_ood_11c_memory_data, dy016_ood_11c_test_data, dy016_ood_11c_test_labels, out_dim=dim)
print(f"DY016 OOD (400 neurons, 200 train spikes) PCA {dim}D GMM score: " + str(dy016_ood_score) + "\n")
dy009_ood_score = pca_gmm_average_tr(fourhund_11c_train_data, dy009_ood_11c_memory_data, dy009_ood_11c_test_data, dy009_ood_11c_test_labels, out_dim=dim)
print(f"DY009 OOD (400 neurons, 200 train spikes) PCA {dim}D GMM score: " + str(dy009_ood_score) + "\n")
testset_A_score = pca_gmm_average_tr(fourhund_11c_train_data, id_11c_memory_data, id_11c_test_data, id_11c_test_labels, out_dim=dim)
print(f"test set A (ID DY016) (400 neurons, 200 train spikes) ID PCA {dim}D GMM score: " + str(testset_A_score) + "\n")

max gmm score: 41.78698411451484
min gmm score: 29.053191512325082
50 run gmm mean score: 34.216320889194364
50 run gmm std-dev score: 2.7071074141174454
DY016 OOD (400 neurons, 200 train spikes) PCA 6D GMM score: 34.216320889194364

max gmm score: 35.442904363780144
min gmm score: 25.594807667495928
50 run gmm mean score: 30.618875993840238
50 run gmm std-dev score: 1.9967235580410598
DY009 OOD (400 neurons, 200 train spikes) PCA 6D GMM score: 30.618875993840238

max gmm score: 45.900629990867614
min gmm score: 32.81812803253475
50 run gmm mean score: 39.09476244850222
50 run gmm std-dev score: 3.673600377855609
test set A (ID DY016) (400 neurons, 200 train spikes) ID PCA 6D GMM score: 39.09476244850222



### 10D

In [33]:
dim = 10
dy016_ood_score = pca_gmm_average_tr(fourhund_denoised_11c_train_data, dy016_denoised_ood_11c_memory_data, dy016_denoised_ood_11c_test_data, dy016_denoised_ood_11c_test_labels, out_dim=dim)
print(f"\nDY016 OOD (390 neurons, 200 train spikes) denoised PCA {dim}D GMM score: " + str(dy016_ood_score) + "\n")
dy009_ood_score = pca_gmm_average_tr(fourhund_denoised_11c_train_data, dy009_denoised_ood_11c_memory_data, dy009_denoised_ood_11c_test_data, dy009_denoised_ood_11c_test_labels, out_dim=dim)
print(f"\nDY009 OOD (390 neurons, 200 train spikes) denoised PCA {dim}D GMM score: " + str(dy009_ood_score) + "\n")
testset_A_score = pca_gmm_average_tr(fourhund_denoised_11c_train_data, id_denoised_11c_memory_data, id_denoised_11c_test_data, id_denoised_11c_test_labels, out_dim=dim)
print(f"\ntest set A (real OOD DY016) (390 neurons, 200 train spikes) denoised PCA {dim}D GMM score: " + str(testset_A_score) + "\n")

max gmm score: 57.18476642724454
min gmm score: 49.883241538378286
50 run gmm mean score: 55.019208284122506
50 run gmm std-dev score: 1.3332919765417302

DY016 OOD (390 neurons, 200 train spikes) denoised PCA 10D GMM score: 55.019208284122506

max gmm score: 57.4905861327079
min gmm score: 42.938224730089765
50 run gmm mean score: 52.120867759237356
50 run gmm std-dev score: 4.016084557095273

DY009 OOD (390 neurons, 200 train spikes) denoised PCA 10D GMM score: 52.120867759237356

max gmm score: 54.50032478734125
min gmm score: 36.82806932996983
50 run gmm mean score: 46.70704239168859
50 run gmm std-dev score: 5.4774460228140684

test set A (real OOD DY016) (390 neurons, 200 train spikes) denoised PCA 10D GMM score: 46.70704239168859



In [34]:
dim = 10
dy016_ood_score = pca_gmm_average_tr(fourhund_11c_train_data, dy016_ood_11c_memory_data, dy016_ood_11c_test_data, dy016_ood_11c_test_labels, out_dim=dim)
print(f"DY016 OOD (400 neurons, 200 train spikes) PCA {dim}D GMM score: " + str(dy016_ood_score) + "\n")
dy009_ood_score = pca_gmm_average_tr(fourhund_11c_train_data, dy009_ood_11c_memory_data, dy009_ood_11c_test_data, dy009_ood_11c_test_labels, out_dim=dim)
print(f"DY009 OOD (400 neurons, 200 train spikes) PCA {dim}D GMM score: " + str(dy009_ood_score) + "\n")
testset_A_score = pca_gmm_average_tr(fourhund_11c_train_data, id_11c_memory_data, id_11c_test_data, id_11c_test_labels, out_dim=dim)
print(f"test set A (ID DY016) (400 neurons, 200 train spikes) ID PCA {dim}D GMM score: " + str(testset_A_score) + "\n")

max gmm score: 40.40950212600583
min gmm score: 28.02585529414987
50 run gmm mean score: 34.274381516991326
50 run gmm std-dev score: 2.5701400210950416
DY016 OOD (400 neurons, 200 train spikes) PCA 10D GMM score: 34.274381516991326

max gmm score: 38.599254391188964
min gmm score: 25.979475407087655
50 run gmm mean score: 32.605137166420484
50 run gmm std-dev score: 3.664304636864035
DY009 OOD (400 neurons, 200 train spikes) PCA 10D GMM score: 32.605137166420484

max gmm score: 48.69308703250956
min gmm score: 37.632748323557
50 run gmm mean score: 42.12261330107361
50 run gmm std-dev score: 2.5117913739183972
test set A (ID DY016) (400 neurons, 200 train spikes) ID PCA 10D GMM score: 42.12261330107361



## 400 neuron, 1200 spikes, 5 chan

In [28]:
dy016_ood_score = pca_gmm_average_tr(fourhund_1200_denoised_5c_train_data, dy016_denoised_ood_5c_memory_data, dy016_denoised_ood_5c_test_data, dy016_denoised_ood_5c_test_labels)
print("DY016 OOD (400 neurons, 1200 train spikes) denoised data GMM score: " + str(dy016_ood_score))
dy009_ood_score = pca_gmm_average_tr(fourhund_1200_denoised_5c_train_data, dy009_denoised_ood_5c_memory_data, dy009_denoised_ood_5c_test_data, dy009_denoised_ood_5c_test_labels)
print("DY009 OOD (400 neurons, 1200 train spikes) denoised data GMM score: " + str(dy009_ood_score))
testset_A_score = pca_gmm_average_tr(fourhund_1200_denoised_5c_train_data, id_denoised_5c_memory_data, id_denoised_5c_test_data, id_denoised_5c_test_labels)
print("test set A (ID DY016) (400 neurons, 1200 train spikes) ID denoised data GMM score: " + str(testset_A_score))

max gmm score: 48.094395698234464
min gmm score: 41.745284448139024
50 run gmm mean score: 44.502718507091146
50 run gmm std-dev score: 1.7779936129471667
DY016 OOD (400 neurons, 1200 train spikes) denoised data GMM score: 44.502718507091146
max gmm score: 44.52775351653091
min gmm score: 34.04123445104575
50 run gmm mean score: 40.33534892495606
50 run gmm std-dev score: 2.6011016533929294
DY009 OOD (400 neurons, 1200 train spikes) denoised data GMM score: 40.33534892495606
max gmm score: 55.92005422904447
min gmm score: 39.23059834144118
50 run gmm mean score: 48.38147559926359
50 run gmm std-dev score: 4.251579451897838
test set A (ID DY016) (400 neurons, 1200 train spikes) ID denoised data GMM score: 48.38147559926359


In [29]:
dy016_ood_score = pca_gmm_average_tr(fourhund_1200_5c_train_data, dy016_ood_5c_memory_data, dy016_ood_5c_test_data, dy016_ood_5c_test_labels)
print("DY016 OOD (400 neurons, 1200 train spikes) data GMM score: " + str(dy016_ood_score))
dy009_ood_score = pca_gmm_average_tr(fourhund_1200_5c_train_data, dy009_ood_5c_memory_data, dy009_ood_5c_test_data, dy009_ood_5c_test_labels)
print("DY009 OOD (400 neurons, 1200 train spikes) denoised data GMM score: " + str(dy009_ood_score))
testset_A_score = pca_gmm_average_tr(fourhund_1200_5c_train_data, id_5c_memory_data, id_5c_test_data, id_5c_test_labels)
print("test set A (ID DY016) (400 neurons, 1200 train spikes) ID data GMM score: " + str(testset_A_score))

max gmm score: 39.89104024339202
min gmm score: 26.546702953463004
50 run gmm mean score: 34.43533178154106
50 run gmm std-dev score: 3.5552293564893485
DY016 OOD (400 neurons, 1200 train spikes) data GMM score: 34.43533178154106
max gmm score: 26.50103263323565
min gmm score: 17.536011558451317
50 run gmm mean score: 20.862068292899586
50 run gmm std-dev score: 1.5991941748639513
DY009 OOD (400 neurons, 1200 train spikes) denoised data GMM score: 20.862068292899586
max gmm score: 51.654971611343306
min gmm score: 35.082526937637624
50 run gmm mean score: 44.87747682661522
50 run gmm std-dev score: 4.001206501321743
test set A (ID DY016) (400 neurons, 1200 train spikes) ID data GMM score: 44.87747682661522


## 400 neuron, 1200 spikes, 11 chan

In [30]:
dy016_ood_score = pca_gmm_average_tr(fourhund_1200_denoised_11c_train_data, dy016_denoised_ood_11c_memory_data, dy016_denoised_ood_11c_test_data, dy016_denoised_ood_11c_test_labels)
print("DY016 OOD (400 neurons, 1200 train spikes) denoised data GMM score: " + str(dy016_ood_score))
dy009_ood_score = pca_gmm_average_tr(fourhund_1200_denoised_11c_train_data, dy009_denoised_ood_11c_memory_data, dy009_denoised_ood_11c_test_data, dy009_denoised_ood_11c_test_labels)
print("DY009 OOD (400 neurons, 1200 train spikes) denoised data GMM score: " + str(dy009_ood_score))
testset_A_score = pca_gmm_average_tr(fourhund_1200_denoised_11c_train_data, id_denoised_11c_memory_data, id_denoised_11c_test_data, id_denoised_11c_test_labels)
print("test set A (ID DY016) (400 neurons, 1200 train spikes) ID denoised data GMM score: " + str(testset_A_score))

max gmm score: 48.99614025165182
min gmm score: 40.78003377881754
50 run gmm mean score: 46.727223066562104
50 run gmm std-dev score: 1.5269670513611084
DY016 OOD (400 neurons, 1200 train spikes) denoised data GMM score: 46.727223066562104
max gmm score: 45.14165294730949
min gmm score: 35.96376483977253
50 run gmm mean score: 41.27244488979042
50 run gmm std-dev score: 2.310532488437097
DY009 OOD (400 neurons, 1200 train spikes) denoised data GMM score: 41.27244488979042
max gmm score: 55.17556987256861
min gmm score: 34.355718816344115
50 run gmm mean score: 44.6044794271138
50 run gmm std-dev score: 4.258011368895061
test set A (ID DY016) (400 neurons, 1200 train spikes) ID denoised data GMM score: 44.6044794271138


In [31]:
dy016_ood_score = pca_gmm_average_tr(fourhund_1200_11c_train_data, dy016_ood_11c_memory_data, dy016_ood_11c_test_data, dy016_ood_11c_test_labels)
print("DY016 OOD (400 neurons, 1200 train spikes) data GMM score: " + str(dy016_ood_score))
dy009_ood_score = pca_gmm_average_tr(fourhund_1200_11c_train_data, dy009_ood_11c_memory_data, dy009_ood_11c_test_data, dy009_ood_11c_test_labels)
print("DY009 OOD (400 neurons, 1200 train spikes) denoised data GMM score: " + str(dy009_ood_score))
testset_A_score = pca_gmm_average_tr(fourhund_1200_11c_train_data, id_11c_memory_data, id_11c_test_data, id_11c_test_labels)
print("test set A (ID DY016) (400 neurons, 1200 train spikes) ID data GMM score: " + str(testset_A_score))

max gmm score: 40.8809364902398
min gmm score: 31.559800924748416
50 run gmm mean score: 36.01833361293192
50 run gmm std-dev score: 2.358189822154822
DY016 OOD (400 neurons, 1200 train spikes) data GMM score: 36.01833361293192
max gmm score: 38.4876234446748
min gmm score: 27.708631557995485
50 run gmm mean score: 30.764194111764947
50 run gmm std-dev score: 2.1523340052588593
DY009 OOD (400 neurons, 1200 train spikes) denoised data GMM score: 30.764194111764947
max gmm score: 43.2844852786714
min gmm score: 26.144529981858213
50 run gmm mean score: 37.77904473049833
50 run gmm std-dev score: 3.780511526098917
test set A (ID DY016) (400 neurons, 1200 train spikes) ID data GMM score: 37.77904473049833


## 390 neuron, 1200 spikes, 11 chan

In [6]:
dy016_ood_score = pca_gmm_average_tr(threenine_1200_denoised_11c_train_data, dy016_denoised_ood_11c_memory_data, dy016_denoised_ood_11c_test_data, dy016_denoised_ood_11c_test_labels)
print("DY016 OOD (390 neurons, 1200 train spikes) denoised data GMM score: " + str(dy016_ood_score))
dy009_ood_score = pca_gmm_average_tr(threenine_1200_denoised_11c_train_data, dy009_denoised_ood_11c_memory_data, dy009_denoised_ood_11c_test_data, dy009_denoised_ood_11c_test_labels)
print("DY009 OOD (390 neurons, 1200 train spikes) denoised data GMM score: " + str(dy009_ood_score))
testset_A_score = pca_gmm_average_tr(threenine_1200_denoised_11c_train_data, id_denoised_11c_memory_data, id_denoised_11c_test_data, id_denoised_11c_test_labels)
print("test set A (real OOD DY016) (390 neurons, 1200 train spikes) denoised data GMM score: " + str(testset_A_score))

max gmm score: 48.60804477338698
min gmm score: 39.75480683937004
50 run gmm mean score: 45.92914436746882
50 run gmm std-dev score: 1.6328892611301953
DY016 OOD (390 neurons, 1200 train spikes) denoised data GMM score: 45.92914436746882
max gmm score: 46.13402195125406
min gmm score: 35.80291780581771
50 run gmm mean score: 42.379070562153146
50 run gmm std-dev score: 2.2303673248404534
DY009 OOD (390 neurons, 1200 train spikes) denoised data GMM score: 42.379070562153146
max gmm score: 55.18234671845307
min gmm score: 35.598123840937454
50 run gmm mean score: 45.77485420791682
50 run gmm std-dev score: 4.1986008009968385
test set A (real OOD DY016) (390 neurons, 1200 train spikes) denoised data GMM score: 45.77485420791682


In [7]:
dy016_ood_score = pca_gmm_average_tr(threenine_1200_11c_train_data, dy016_ood_11c_memory_data, dy016_ood_11c_test_data, dy016_ood_11c_test_labels)
print("DY016 OOD (390 neurons, 1200 train spikes) data GMM score: " + str(dy016_ood_score))
dy009_ood_score = pca_gmm_average_tr(threenine_1200_11c_train_data, dy009_ood_11c_memory_data, dy009_ood_11c_test_data, dy009_ood_11c_test_labels)
print("DY009 OOD (390 neurons, 1200 train spikes) denoised data GMM score: " + str(dy009_ood_score))
testset_A_score = pca_gmm_average_tr(threenine_1200_11c_train_data, id_11c_memory_data, id_11c_test_data, id_11c_test_labels)
print("test set A (real OOD DY016) (390 neurons, 1200 train spikes) data GMM score: " + str(testset_A_score))

max gmm score: 39.189555871252686
min gmm score: 28.935972677835114
50 run gmm mean score: 34.53771812976084
50 run gmm std-dev score: 2.1985301275746054
DY016 OOD (390 neurons, 1200 train spikes) data GMM score: 34.53771812976084
max gmm score: 36.17186157242666
min gmm score: 27.277896826425675
50 run gmm mean score: 31.420735723586652
50 run gmm std-dev score: 2.175408313716692
DY009 OOD (390 neurons, 1200 train spikes) denoised data GMM score: 31.420735723586652
max gmm score: 43.24709337992005
min gmm score: 32.93959869314438
50 run gmm mean score: 37.9835405610387
50 run gmm std-dev score: 3.344276432386825
test set A (real OOD DY016) (390 neurons, 1200 train spikes) data GMM score: 37.9835405610387


## 390 neuron, 1200 spikes, 5 chan

In [9]:
# dy016_ood_score = pca_gmm_average_tr(threenine_1200_denoised_5c_train_data, dy016_denoised_ood_5c_memory_data, dy016_denoised_ood_5c_test_data, dy016_denoised_ood_5c_test_labels)
# print("DY016 OOD (390 neurons, 1200 train spikes) denoised data GMM score: " + str(dy016_ood_score))
# dy009_ood_score = pca_gmm_average_tr(threenine_1200_denoised_5c_train_data, dy009_denoised_ood_5c_memory_data, dy009_denoised_ood_5c_test_data, dy009_denoised_ood_5c_test_labels)
# print("DY009 OOD (390 neurons, 1200 train spikes) denoised data GMM score: " + str(dy009_ood_score))
testset_A_score = pca_gmm_average_tr(threenine_1200_denoised_5c_train_data, id_denoised_5c_memory_data, id_denoised_5c_test_data, id_denoised_5c_test_labels)
print("test set A (real OOD DY016) (390 neurons, 1200 train spikes) denoised data GMM score: " + str(testset_A_score))

max gmm score: 55.549688984912414
min gmm score: 41.62956135000746
50 run gmm mean score: 49.46433902144398
50 run gmm std-dev score: 4.305482778200534
test set A (real OOD DY016) (390 neurons, 1200 train spikes) denoised data GMM score: 49.46433902144398


In [10]:
# dy016_ood_score = pca_gmm_average_tr(threenine_1200_5c_train_data, dy016_ood_5c_memory_data, dy016_ood_5c_test_data, dy016_ood_5c_test_labels)
# print("DY016 OOD (390 neurons, 1200 train spikes) data GMM score: " + str(dy016_ood_score))
# dy009_ood_score = pca_gmm_average_tr(threenine_1200_5c_train_data, dy009_ood_5c_memory_data, dy009_ood_5c_test_data, dy009_ood_5c_test_labels)
# print("DY009 OOD (390 neurons, 1200 train spikes) denoised data GMM score: " + str(dy009_ood_score))
testset_A_score = pca_gmm_average_tr(threenine_1200_5c_train_data, id_5c_memory_data, id_5c_test_data, id_5c_test_labels)
print("test set A (real OOD DY016) (390 neurons, 1200 train spikes) data GMM score: " + str(testset_A_score))

max gmm score: 50.04236190276502
min gmm score: 35.211012333715445
50 run gmm mean score: 44.65907862400737
50 run gmm std-dev score: 3.1798425257429526
test set A (real OOD DY016) (390 neurons, 1200 train spikes) data GMM score: 44.65907862400737


## 390 neuron, 200 spikes, 11 chan

In [13]:
# dy016_ood_score = pca_gmm_average_tr(threenine_200_denoised_11c_train_data, dy016_denoised_ood_11c_memory_data, dy016_denoised_ood_11c_test_data, dy016_denoised_ood_11c_test_labels)
# print("DY016 OOD (390 neurons, 200 train spikes) denoised data GMM score: " + str(dy016_ood_score))
# dy009_ood_score = pca_gmm_average_tr(threenine_200_denoised_11c_train_data, dy009_denoised_ood_11c_memory_data, dy009_denoised_ood_11c_test_data, dy009_denoised_ood_11c_test_labels)
# print("DY009 OOD (390 neurons, 200 train spikes) denoised data GMM score: " + str(dy009_ood_score))
testset_A_score = pca_gmm_average_tr(threenine_200_denoised_11c_train_data, id_denoised_11c_memory_data, id_denoised_11c_test_data, id_denoised_11c_test_labels)
print("test set A (real OOD DY016) (390 neurons, 200 train spikes) denoised data GMM score: " + str(testset_A_score))

max gmm score: 54.142899883337655
min gmm score: 35.40300261482869
50 run gmm mean score: 43.8595789884621
50 run gmm std-dev score: 3.607852687923732
test set A (real OOD DY016) (390 neurons, 200 train spikes) denoised data GMM score: 43.8595789884621


In [14]:
# dy016_ood_score = pca_gmm_average_tr(threenine_200_11c_train_data, dy016_ood_11c_memory_data, dy016_ood_11c_test_data, dy016_ood_11c_test_labels)
# print("DY016 OOD (390 neurons, 200 train spikes) data GMM score: " + str(dy016_ood_score))
# dy009_ood_score = pca_gmm_average_tr(threenine_200_11c_train_data, dy009_ood_11c_memory_data, dy009_ood_11c_test_data, dy009_ood_11c_test_labels)
# print("DY009 OOD (390 neurons, 200 train spikes) denoised data GMM score: " + str(dy009_ood_score))
testset_A_score = pca_gmm_average_tr(threenine_200_11c_train_data, id_11c_memory_data, id_11c_test_data, id_11c_test_labels)
print("test set A (real OOD DY016) (390 neurons, 200 train spikes) data GMM score: " + str(testset_A_score))

max gmm score: 44.75409980235697
min gmm score: 33.64156989652088
50 run gmm mean score: 40.080611925455344
50 run gmm std-dev score: 3.2865079971507436
test set A (real OOD DY016) (390 neurons, 200 train spikes) data GMM score: 40.080611925455344


## 390 neuron, 200 spikes, 5 chan

In [18]:
# dy016_ood_score = pca_gmm_average_tr(threenine_200_denoised_5c_train_data, dy016_denoised_ood_5c_memory_data, dy016_denoised_ood_5c_test_data, dy016_denoised_ood_5c_test_labels)
# print("DY016 OOD (390 neurons, 200 train spikes) denoised data GMM score: " + str(dy016_ood_score))
# dy009_ood_score = pca_gmm_average_tr(threenine_200_denoised_5c_train_data, dy009_denoised_ood_5c_memory_data, dy009_denoised_ood_5c_test_data, dy009_denoised_ood_5c_test_labels)
# print("DY009 OOD (390 neurons, 200 train spikes) denoised data GMM score: " + str(dy009_ood_score))
testset_A_score = pca_gmm_average_tr(threenine_200_denoised_5c_train_data, id_denoised_5c_memory_data, id_denoised_5c_test_data, id_denoised_5c_test_labels)
print("test set A (real OOD DY016) (390 neurons, 200 train spikes) denoised data GMM score: " + str(testset_A_score))

max gmm score: 54.48242513321132
min gmm score: 40.95370606120831
50 run gmm mean score: 47.71489214919725
50 run gmm std-dev score: 4.254024821979045
test set A (real OOD DY016) (390 neurons, 200 train spikes) denoised data GMM score: 47.71489214919725


In [19]:
# dy016_ood_score = pca_gmm_average_tr(threenine_1200_5c_train_data, dy016_ood_5c_memory_data, dy016_ood_5c_test_data, dy016_ood_5c_test_labels)
# print("DY016 OOD (390 neurons, 1200 train spikes) data GMM score: " + str(dy016_ood_score))
# dy009_ood_score = pca_gmm_average_tr(threenine_1200_5c_train_data, dy009_ood_5c_memory_data, dy009_ood_5c_test_data, dy009_ood_5c_test_labels)
# print("DY009 OOD (390 neurons, 1200 train spikes) denoised data GMM score: " + str(dy009_ood_score))
testset_A_score = pca_gmm_average_tr(threenine_200_5c_train_data, id_5c_memory_data, id_5c_test_data, id_5c_test_labels)
print("test set A (real OOD DY016) (390 neurons, 200 train spikes) data GMM score: " + str(testset_A_score))

max gmm score: 50.26284972490571
min gmm score: 35.28060336664095
50 run gmm mean score: 43.43548326743042
50 run gmm std-dev score: 3.7924848032514658
test set A (real OOD DY016) (390 neurons, 200 train spikes) data GMM score: 43.43548326743042


## ID of all sets

In [9]:
dy016_ood_score = pca_gmm_average(dy016_denoised_ood_5c_memory_data, dy016_denoised_ood_5c_test_data, dy016_denoised_ood_5c_test_labels)
print("DY016 ID (200 train spikes) denoised data GMM score: " + str(dy016_ood_score))
dy009_ood_score = pca_gmm_average(dy009_denoised_ood_5c_memory_data, dy009_denoised_ood_5c_test_data, dy009_denoised_ood_5c_test_labels)
print("DY009 ID (200 train spikes) denoised data GMM score: " + str(dy009_ood_score))
testset_A_score = pca_gmm_average(id_denoised_5c_memory_data, id_denoised_5c_test_data, id_denoised_5c_test_labels)
print("test set A (DY016) (200 train spikes) ID denoised data GMM score: " + str(testset_A_score))

max gmm score: 52.584032309117376
min gmm score: 46.0081820452671
50 run gmm mean score: 49.2891235247913
50 run gmm std-dev score: 1.6740202597724116
DY016 ID (200 train spikes) denoised data GMM score: 49.2891235247913
max gmm score: 47.609536052231974
min gmm score: 38.36627362684138
50 run gmm mean score: 43.84671865114465
50 run gmm std-dev score: 2.4790120498438704
DY009 ID (200 train spikes) denoised data GMM score: 43.84671865114465
max gmm score: 55.13788604316496
min gmm score: 41.24540178404644
50 run gmm mean score: 50.089815808558384
50 run gmm std-dev score: 3.7784439320488157
test set A (DY016) (200 train spikes) ID denoised data GMM score: 50.089815808558384


In [10]:
dy016_ood_score = pca_gmm_average(dy016_denoised_ood_11c_memory_data, dy016_denoised_ood_11c_test_data, dy016_denoised_ood_11c_test_labels)
print("DY016 ID (200 train spikes) denoised data GMM score: " + str(dy016_ood_score))
dy009_ood_score = pca_gmm_average(dy009_denoised_ood_11c_memory_data, dy009_denoised_ood_11c_memory_data, dy009_denoised_ood_11c_test_labels)
print("DY009 ID (200 train spikes) denoised data GMM score: " + str(dy009_ood_score))
testset_A_score = pca_gmm_average(id_denoised_11c_memory_data, id_denoised_11c_test_data, id_denoised_11c_test_labels)
print("test set A (DY016) (200 train spikes) ID data denoised GMM score: " + str(testset_A_score))

max gmm score: 52.87893863727135
min gmm score: 46.19428913641999
50 run gmm mean score: 49.9818882433913
50 run gmm std-dev score: 1.3541427496136573
DY016 ID (200 train spikes) denoised data GMM score: 49.9818882433913
max gmm score: 49.72548301103939
min gmm score: 44.87138629207036
50 run gmm mean score: 48.07065613084811
50 run gmm std-dev score: 1.503227640182234
DY009 ID (200 train spikes) denoised data GMM score: 48.07065613084811
max gmm score: 44.28033538250909
min gmm score: 31.39884573758237
50 run gmm mean score: 39.61758101381981
50 run gmm std-dev score: 2.7179886027734343
test set A (DY016) (200 train spikes) ID data denoised GMM score: 39.61758101381981


In [7]:
dy016_ood_score = pca_gmm_average(dy016_ood_5c_memory_data, dy016_ood_5c_test_data, dy016_ood_5c_test_labels)
print("DY016 ID (200 train spikes) data GMM score: " + str(dy016_ood_score))
dy009_ood_score = pca_gmm_average(dy009_ood_5c_memory_data, dy009_ood_5c_memory_data, dy009_ood_5c_test_labels)
print("DY009 ID (200 train spikes) data GMM score: " + str(dy009_ood_score))
testset_A_score = pca_gmm_average(id_5c_memory_data, id_5c_test_data, id_5c_test_labels)
print("test set A (DY016) (200 train spikes) ID data GMM score: " + str(testset_A_score))

max gmm score: 44.016484964451244
min gmm score: 34.27168700179398
50 run gmm mean score: 38.60536750922742
50 run gmm std-dev score: 2.2499194442618533
DY016 OOD data GMM score: 38.60536750922742
max gmm score: 43.2123786410486
min gmm score: 32.315591872083886
50 run gmm mean score: 39.17478255524215
50 run gmm std-dev score: 2.2927240799944033
DY009 OOD data GMM score: 39.17478255524215
max gmm score: 54.85513270593066
min gmm score: 36.60393970101378
50 run gmm mean score: 44.16644158201513
50 run gmm std-dev score: 3.601146433041381
test set A data GMM score: 44.16644158201513


In [8]:
dy016_ood_score = pca_gmm_average(dy016_ood_11c_memory_data, dy016_ood_11c_test_data, dy016_ood_11c_test_labels)
print("DY016 ID (200 train spikes) data GMM score: " + str(dy016_ood_score))
dy009_ood_score = pca_gmm_average(dy009_ood_11c_memory_data, dy009_ood_11c_memory_data, dy009_ood_11c_test_labels)
print("DY009 ID (200 train spikes) data GMM score: " + str(dy009_ood_score))
testset_A_score = pca_gmm_average(id_11c_memory_data, id_11c_test_data, id_11c_test_labels)
print("test set A (DY016) (200 train spikes) ID data GMM score: " + str(testset_A_score))

max gmm score: 41.339826356672376
min gmm score: 29.16540999756477
50 run gmm mean score: 36.14958006262585
50 run gmm std-dev score: 2.557504424154209
DY016 OOD data GMM score: 36.14958006262585
max gmm score: 40.11572258482997
min gmm score: 30.17548036088694
50 run gmm mean score: 36.8479892578336
50 run gmm std-dev score: 2.278244767294241
DY009 OOD data GMM score: 36.8479892578336
max gmm score: 47.95558042099082
min gmm score: 33.61906273944858
50 run gmm mean score: 41.315006775805585
50 run gmm std-dev score: 3.819610125152901
test set A data GMM score: 41.315006775805585


In [12]:
dy016_ood_score = pca_gmm_average(dy016_id_5c_memory_data, dy016_ood_5c_test_data, dy016_ood_5c_test_labels)
print("DY016 OOD (test set A 1200 spikes) data GMM score: " + str(dy016_ood_score))
dy009_ood_score = pca_gmm_average(dy016_id_5c_memory_data, dy009_ood_5c_memory_data, dy009_ood_5c_test_labels)
print("DY009 OOD (test set A 1200 spikes) data GMM score: " + str(dy009_ood_score))
testset_A_score = pca_gmm_average(dy016_id_5c_memory_data, id_5c_test_data, id_5c_test_labels)
print("test set A (test set A 1200 spikes) data GMM score: " + str(testset_A_score))

max gmm score: 24.715127816076123
min gmm score: 20.984923304102924
50 run gmm mean score: 23.58112385071734
50 run gmm std-dev score: 0.5275223408566994
DY016 OOD (test set A 1200 spikes) data GMM score: 23.58112385071734
max gmm score: 21.238489666225092
min gmm score: 14.593883750991893
50 run gmm mean score: 17.9354498400044
50 run gmm std-dev score: 0.9226892008696899
DY009 OOD (test set A 1200 spikes) data GMM score: 17.9354498400044
max gmm score: 48.48341232339838
min gmm score: 42.088143716207256
50 run gmm mean score: 46.35456126187191
50 run gmm std-dev score: 2.02502218740775
test set A (test set A 1200 spikes) data GMM score: 46.35456126187191


In [13]:
dy016_ood_score = pca_gmm_average(dy016_id_11c_memory_data, dy016_ood_11c_test_data, dy016_ood_11c_test_labels)
print("DY016 OOD (test set A 1200 spikes) data GMM score: " + str(dy016_ood_score))
dy009_ood_score = pca_gmm_average(dy016_id_11c_memory_data, dy009_ood_11c_memory_data, dy009_ood_11c_test_labels)
print("DY009 OOD (test set A 1200 spikes) data GMM score: " + str(dy009_ood_score))
testset_A_score = pca_gmm_average(dy016_id_11c_memory_data, id_11c_test_data, id_11c_test_labels)
print("test set A (test set A 1200 spikes) data GMM score: " + str(testset_A_score))

max gmm score: 23.662534858583182
min gmm score: 10.746395164699681
50 run gmm mean score: 16.4852522192692
50 run gmm std-dev score: 3.3836952638107967
DY016 OOD (test set A 1200 spikes) data GMM score: 16.4852522192692
max gmm score: 22.049626883106228
min gmm score: 9.837814120694349
50 run gmm mean score: 15.901064382157278
50 run gmm std-dev score: 4.077840670932784
DY009 OOD (test set A 1200 spikes) data GMM score: 15.901064382157278
max gmm score: 43.13007028785448
min gmm score: 28.370391755074348
50 run gmm mean score: 36.29547030978494
50 run gmm std-dev score: 3.6242172401098993
test set A (test set A 1200 spikes) data GMM score: 36.29547030978494
